R environment

In [ ]:
library(tidyverse)
library(Seurat)
library(stringr)
library(tibble)
library(data.table)
library(DoubletFinder)
library(KernSmooth)
library(Matrix)
library(dplyr)
library(future)
library(cluster)
library(HGNChelper)
library(parallel)
source("helper_functions_v1.R")
library(future)
set.seed(1)
plan("multiprocess", workers = 8)
options(future.globals.maxSize = 1000 * 1024^6)

Filtering criteria: <br>
- min.cells = 1000 per study <br>
- min.gene = 500 <br>
- min.umi = 1000 <br>
- max.MT = 30 <br>

In [ ]:
path.dir = '/hpc/pmc_stunnenberg/cruiz/scRNA/markers-and-databases/GBM_public-data/'

Clinical data abbreviations

__WT__=wild type  <br>
__NT__=not tested  <br>
__M__=promoter methylated  <br> 
__NM__=promoter not methylated

## Studies focused on malignant cells

### Yuan2018
Raw counts matrices <br>
Microwell-seq (custom platform) <br>
Unkwon whether Matrices were filtered cells by QC (using TISHC metadata for filtering)

In [ ]:
tidy.tables <- function(filename) {
    dat=data.table::fread(filename)
    dat <- as.data.frame(dat[,-1])
    dat <- dat[!duplicated(dat$V2), ]
    rownames(dat) <- NULL
    dat <- column_to_rownames(dat, 'V2')
    colnames(dat) <- seq(0, length(colnames(dat))-1)
    dat <- as.matrix(dat)
}

In [ ]:
tmp <- list.files(paste0(path.dir, "Yuan2018"),
               pattern = "*.txt.gz", 
               full.names = T)
myfiles = lapply(tmp, tidy.tables)

In [ ]:
labels <- str_sub(tmp, start = 92, end=str_locate(tmp,"filtered")[,1]-2)
names(myfiles) <- labels

In [ ]:
Yuan2018 <- list()
for (i in 1:length(myfiles)) {
    seu_obj <- CreateSeuratObject(myfiles[[i]], min.features = 500, min.cells = 5,
                                  project = names(myfiles[i]))
    Yuan2018 <- append(Yuan2018, seu_obj)
}

In [ ]:
Yuan2018_final <- merge(Yuan2018[[1]], y = unlist(Yuan2018[2:length(Yuan2018)]), 
             add.cell.ids = labels, project = "Yuan2018", merge.data = T)

In [ ]:
Yuan2018_final <- PercentageFeatureSet(Yuan2018_final, pattern = "^MT-", col.name = "percent.MT")
Yuan2018_final <- subset(Yuan2018_final, subset = percent.MT < 30 & nCount_RNA > 1000)

In [ ]:
# clinical metadata
Yuan2018_clinical <- subset(data.table::fread(paste0(path.dir, 'Yuan2018/clinical_data.txt')),
                               select = -c(IDH, `Diagnosis`))
Yuan2018_clinical <- Yuan2018_clinical %>% subset(!(Patient == 'PJ016' | # this patient is IDH mutant
                                         Patient == 'PJ030')) # this patient had anaplastic astrocytoma

In [ ]:
# no original metadata available
Yuan2018_meta <- subset(data.table::fread(paste0(path.dir, 'Yuan2018/GBM_GSE103224_CellMetainfo_table.tsv')),
                        select = -c(UMAP_1,UMAP_2,Cluster))
Yuan2018_meta <- Yuan2018_meta %>% subset(!(Patient == 'PJ016' | # this patient is IDH mutant
                                         Patient == 'PJ030')) # this patient had anaplastic astrocytoma

In [ ]:
Yuan2018_meta <- merge(Yuan2018_meta, Yuan2018_clinical, by= "Patient")
Yuan2018_meta$Platform <- 'Microwell-seq'
Yuan2018_meta$Method <- 'cell'

In [ ]:
Yuan2018_matrix <- GetAssayData(Yuan2018_final, slot = 'counts')[,colnames(Yuan2018_final) %in% Yuan2018_meta$Cell]

In [ ]:
gene.names <- checkGeneSymbols(rownames(Yuan2018_matrix), unmapped.as.na=FALSE)
rownames(Yuan2018_matrix) <- make.unique(gene.names$Suggested.Symbol)

### Neftel2019

#### Smart-seq2 - log2(TPM/10+1) values <br>
Matrices include only filtered cells by QC

In [ ]:
Neftel2019_smart <- data.table::fread(paste0(path.dir, 'Neftel2019/IDHwtGBM.processed.SS2.logTPM.txt.gz')) %>% 
                    column_to_rownames('GENE')

In [ ]:
# convert from log2(TPM/10+1) to TPM
matrix <- as.matrix(Neftel2019_smart)
converted_matrix <- (2^matrix-1) * 10
original_matrix <- log2(converted_matrix/10 + 1)
all.equal(matrix,original_matrix) # verify that the conversion gives the expected results
Neftel2019_smart <- round(converted_matrix)

In [ ]:
Neftel2019_smart_final <- CreateSeuratObject(Neftel2019_smart, min.features = 500, min.cells = 5, 
                                  project = 'Neftel2019_smart')

In [ ]:
Neftel2019_smart_final <- PercentageFeatureSet(Neftel2019_smart_final, pattern = "^MT-", col.name = "percent.MT")
Neftel2019_smart_final <- subset(Neftel2019_smart_final, subset = percent.MT < 30)

In [ ]:
# clinical metadata
Neftel2019_clinical <- subset(data.table::fread(paste0(path.dir, 'Neftel2019/clinical_data.txt')),
                             select = -c(IDH, `Other mutations`))
Neftel2019_clinical$EGFR[Neftel2019_clinical$EGFR == "Positive"] <- "amplified"
Neftel2019_clinical[Neftel2019_clinical == "Methylated"] <- "M"
colnames(Neftel2019_clinical)[1] <- 'Patient'

In [ ]:
Neftel2019_smart_meta <- data.table::fread(paste0(path.dir, 'Neftel2019/GBM_GSE131928_Smartseq2_CellMetainfo_table.tsv'))

In [ ]:
Neftel2019_smart_meta <- Neftel2019_smart_meta %>% filter(AgeGroup == 'Adult') # do not include pediatric samples
Neftel2019_smart_meta <- subset(Neftel2019_smart_meta, select = -c(UMAP_1,UMAP_2,Cluster, AgeGroup))

In [ ]:
Neftel2019_smart_meta <- merge(Neftel2019_smart_meta, Neftel2019_clinical, by= "Patient")
Neftel2019_smart_meta$Platform <- 'Smart-seq2'
Neftel2019_smart_meta$Method <- 'cell'

In [ ]:
Neftel2019_smart_matrix <- GetAssayData(Neftel2019_smart_final, slot = 'counts')[,colnames(Neftel2019_smart_final) %in%  Neftel2019_smart_meta$Cell]

In [ ]:
gene.names <- checkGeneSymbols(rownames(Neftel2019_smart_matrix), unmapped.as.na=FALSE)
rownames(Neftel2019_smart_matrix) <- make.unique(gene.names$Suggested.Symbol)

#### 10x genomics - raw counts <br>
Matrices include only filtered cells by QC

In [ ]:
Neftel2019_10x_1 <- CreateSeuratObject(Read10X(paste0(path.dir, '/Neftel2019/scRNA_10x/IDHwtGBM_1/')),
                                       min.features = 500, min.cells = 5, project = 'Neftel2019_10x_1')
Neftel2019_10x_2 <- CreateSeuratObject(Read10X(paste0(path.dir, '/Neftel2019/scRNA_10x/IDHwtGBM_2/')), 
                                       min.features = 500, min.cells = 5, project = 'Neftel2019_10x_2')
merged_Neftel <- merge(Neftel2019_10x_1, Neftel2019_10x_2)

In [ ]:
# matrices contain different samples. First, they are split by orig.ident
seu.list <- list()
for(i in 1:length(table(merged_Neftel@meta.data$orig.ident))){
    seu_obj <- subset(merged_Neftel, subset = orig.ident == names(table(merged_Neftel@meta.data$orig.ident))[[i]])
    seu.list <- append(seu.list, seu_obj)
    }

In [ ]:
for(i in 1:length(seu.list)){
  cat(' #####################################\n',
      '### Processing dataset number ', i, '###\n',
      '#####################################\n')
  # add %MT
  seu.list[[i]][["percent.MT"]]  <- PercentageFeatureSet(seu.list[[i]], pattern = "^MT-") 
  
  # Filter out low quality cells according to the metrics defined above
  seu.list[[i]] <- subset(seu.list[[i]],
                           subset = percent.MT < 30 &
                             nCount_RNA > 1000
  )
  # Only mito and floor filtering; trying to find doublets
}

# preprocess each dataset individually
seu.list <- lapply(seu.list, seuPreProcess)

In [ ]:
# https://github.com/mckellardw/scMuscle/blob/main/R_scripts/scMuscle_github_v1.R

bcmvn <- list()
pK <- list()
homotypic.prop <- list()
nExp_poi <- list()
nExp_poi.adj <- list()

# Estimated Doublet Rate for each dataset
edr <- estimateDoubletRate.DWM(seur.list = seu.list)/100 #use your own known EDR here

for(i in 1:length(seu.list)){
  cat(' ############################################\n',
      '### DoubletFinder for dataset number ', i, '###\n',
      '############################################\n')
  
  ## pK Identification (no ground-truth)
  bcmvn[[i]]<- paramSweep_v3(
    seu=seu.list[[i]],
    PCs = 1:seu.list[[i]]@reductions$umap_RNA@misc$n.pcs.used, 
    num.cores = 8
  ) %>% summarizeSweep(
    GT = FALSE
  ) %>% find.pK() 
  
  # Pull out max of bcmvn
  pK[[i]] <- as.numeric(as.character(bcmvn[[i]]$pK[bcmvn[[i]]$BCmetric==max(bcmvn[[i]]$BCmetric)])) # ugly, but functional...
  
  ## Homotypic Doublet Proportion Estimate
  homotypic.prop[[i]] <- modelHomotypic(seu.list[[i]]$seurat_clusters) 
  
  nExp_poi[[i]] <- round(edr[[i]]*length(colnames(seu.list[[i]])))  
  nExp_poi.adj[[i]] <- round(nExp_poi[[i]]*(1-homotypic.prop[[i]]))
}

In [ ]:
# Run DoubletFinder
for(i in 1:length(seu.list)){
  seu.list[[i]] <- 
    doubletFinder_V3.DWM_v2( # just changed it so the output metadata column name is customizable
      seu=seu.list[[i]], 
      PCs = 1:seu.list[[i]]@reductions$umap_RNA@misc$n.pcs.used, 
      pN = 0.25, #default value
      pK= pK[[i]], 
      nExp = nExp_poi.adj[[i]],  
      reuse.pANN = F, 
      classification.name='DF.individual', 
      pANN.name='DF.pANN.individual'
    )
}

In [ ]:
Neftel2019_10x_final <- merge(seu.list[[1]], y = unlist(seu.list[2:length(seu.list)]), 
              project = "Neftel2019_10x", merge.data = T) # it is not necessary to add cell IDs here because they are already in the name of each cell

In [ ]:
# filter doublets
Neftel2019_10x_final <- subset(Neftel2019_10x_final, subset = DF.individual == 'Singlet')

In [ ]:
Neftel2019_10x_meta <- subset(data.table::fread(paste0(path.dir, 'Neftel2019/GBM_GSE131928_10X_CellMetainfo_table.tsv')),
                              select = -c(UMAP_1,UMAP_2,Cluster))

In [ ]:
Neftel2019_10x_meta <- merge(Neftel2019_10x_meta, Neftel2019_clinical, 
                             by= "Patient", all = TRUE)
Neftel2019_10x_meta[Neftel2019_10x_meta$Cell %in% colnames(Neftel2019_10x_final) ,]
Neftel2019_10x_meta$Platform <- '10x_v2'
Neftel2019_10x_meta$Method <- 'cell'

In [ ]:
Neftel2019_10x_matrix <- GetAssayData(Neftel2019_10x_final, slot = 'counts')[,colnames(Neftel2019_10x_final) %in%  Neftel2019_10x_meta$Cell]

In [ ]:
gene.names <- checkGeneSymbols(rownames(Neftel2019_10x_matrix), unmapped.as.na=FALSE)
rownames(Neftel2019_10x_matrix) <- make.unique(gene.names$Suggested.Symbol)

### Wang2019
Raw counts matrices<br>
10x Genomics platform

In [ ]:
tmp <- list.files(path = paste0(path.dir, "Wang2019"), pattern = "^SF",
               full.names = T)

In [ ]:
labels <- str_sub(tmp, start = 81) 
names(tmp) <- labels

In [ ]:
Wang2019 <- list()
for (i in 1:length(tmp)) {
    seu_obj.data <- Read10X(data.dir = tmp[[i]], gene.column = 1)
    seu_obj <- CreateSeuratObject(seu_obj.data, min.features = 0, min.cells = 5, 
                                  project = names(tmp[i]))
    Wang2019 <- append(Wang2019, seu_obj)
}

In [ ]:
# preprocess each dataset individually
Wang2019 <- lapply(Wang2019, seuPreProcess)

In [ ]:
# https://github.com/mckellardw/scMuscle/blob/main/R_scripts/scMuscle_github_v1.R

bcmvn <- list()
pK <- list()
homotypic.prop <- list()
nExp_poi <- list()
nExp_poi.adj <- list()

# Estimated Doublet Rate for each dataset
edr <- estimateDoubletRate.DWM(seur.list = Wang2019)/100 #use your own known EDR here

for(i in 1:length(Wang2019)){
  cat(' ############################################\n',
      '### DoubletFinder for dataset number ', i, '###\n',
      '############################################\n')
  
  ## pK Identification (no ground-truth)
  bcmvn[[i]]<- paramSweep_v3(
    seu=Wang2019[[i]],
    PCs = 1:Wang2019[[i]]@reductions$umap_RNA@misc$n.pcs.used, 
    num.cores = 8
  ) %>% summarizeSweep(
    GT = FALSE
  ) %>% find.pK() 
  
  # Pull out max of bcmvn
  pK[[i]] <- as.numeric(as.character(bcmvn[[i]]$pK[bcmvn[[i]]$BCmetric==max(bcmvn[[i]]$BCmetric)])) # ugly, but functional...
  
  ## Homotypic Doublet Proportion Estimate
  homotypic.prop[[i]] <- modelHomotypic(Wang2019[[i]]$seurat_clusters) 
  
  nExp_poi[[i]] <- round(edr[[i]]*length(colnames(Wang2019[[i]])))  
  nExp_poi.adj[[i]] <- round(nExp_poi[[i]]*(1-homotypic.prop[[i]]))
}

In [ ]:
# Run DoubletFinder
for(i in 1:length(Wang2019)){
  Wang2019[[i]] <- 
    doubletFinder_V3.DWM_v2( # just changed it so the output metadata column name is customizable
      seu=Wang2019[[i]], 
      PCs = 1:Wang2019[[i]]@reductions$umap_RNA@misc$n.pcs.used, 
      pN = 0.25, #default value
      pK= pK[[i]], 
      nExp = nExp_poi.adj[[i]],  
      reuse.pANN = F, 
      classification.name='DF.individual', 
      pANN.name='DF.pANN.individual'
    )
}

In [ ]:
Wang2019_final <- merge(Wang2019[[1]], y = unlist(Wang2019[2:length(Wang2019)]), 
              add.cell.ids = labels, project = "Wang2019", merge.data = T)

In [ ]:
Wang2019_final <- PercentageFeatureSet(Wang2019_final, pattern = "^MT-", col.name = "percent.MT")
Wang2019_final <- subset(Wang2019_final, subset = percent.MT < 30 & nCount_RNA > 1000 & nFeature_RNA > 500)

In [ ]:
# filter doublets
Wang2019_final <- subset(Wang2019_final, subset = DF.individual == 'Singlet')

In [ ]:
# clinical metadata
Wang2019_clinical <- subset(data.table::fread(paste0(path.dir, 'Wang2019/clinical_data.txt')),
                            select = -c(`IDH status`))

In [ ]:
# TISCH included several samples that were not GBM IDHwt (Oligodendroglioma, astrocytoma)
# metadata paper here https://cancerdiscovery.aacrjournals.org/content/9/12/1708.figures-only

Wang2019_meta <- data.table::fread(paste0(path.dir, 'Wang2019/GBM_GSE138794_CellMetainfo_table.tsv'))
table(as.factor(Wang2019_meta$Patient))
Wang2019_meta$Cell <- gsub('@', '_', Wang2019_meta$Cell) # change characters to match cell names
Wang2019_meta <- subset(Wang2019_meta, select = -c(UMAP_1,UMAP_2,Cluster)) %>% filter(Patient == 'SF11956' |
                                                                                     Patient == 'SF11977' |
                                                                                     Patient == 'SF11644' |
                                                                                     Patient == 'SF11979' )
                # Filter patients that are only GBM IDHwt

In [ ]:
# there is only metadata for the single nuclei profiles (not single cells)
Wang2019_meta_original <- data.table::fread(paste0(path.dir, 'Wang2019/GSE138794_snRNA_Seq_cell_types.txt'),
                                           header = F)
colnames(Wang2019_meta_original) <- c('Cell', 'Celltype (original)')

In [ ]:
patients <- str_sub(Wang2019_meta_original$Cell, start = 0, end=str_locate(Wang2019_meta_original$Cell,"_")[,1]-1)
Wang2019_meta_original$Patient <- patients

In [ ]:
Wang2019_meta <- Wang2019_meta %>% full_join(Wang2019_meta_original, by = c("Cell", "Patient"))
Wang2019_meta <- Wang2019_meta %>% filter(!Patient == 'SF12090' & !Patient == 'SF12286') # SF12286 does not exists (neither in raw data nor supp metadata)

In [ ]:
Wang2019_meta <- merge(Wang2019_meta, Wang2019_clinical, by= "Patient")
Wang2019_meta$Platform <- '10x_v2'

In [ ]:
Wang2019_matrix <- GetAssayData(Wang2019_final,  slot = 'counts')[,colnames(Wang2019_final) %in% Wang2019_meta$Cell]

In [ ]:
gene.names <- checkGeneSymbols(rownames(Wang2019_matrix), unmapped.as.na=FALSE)
rownames(Wang2019_matrix) <- make.unique(gene.names$Suggested.Symbol)

### Wang2020
10x Genomics - Raw counts matrices <br>
Unkwon whether Matrices were filtered cells by QC (using TISHC metadata for filtering)

In [ ]:
tidy.tables <- function(filename) {
    dat <- read.table(filename, sep = ",") %>% t()
    colnames(dat) <- dat[1, ]
    colnames(dat) <- make.names(colnames(dat), unique = TRUE)
    dat <- as.data.frame(dat)
    dat <- dat[!duplicated(dat$NA.), ]
    dat <- dat[-1:-2, ]
    rownames(dat) <- NULL
    dat <- column_to_rownames(dat, "NA.")
    cols <- gsub("e.", "e+", colnames(dat))
    colnames(dat) <- cols
    dat <- as.matrix(dat)
}

In [ ]:
tmp <- list.files(path = paste0(path.dir, "Wang2020"),
               pattern = "*.csv.gz", 
               full.names = T)
myfiles = lapply(tmp, tidy.tables))

In [ ]:
labels <- str_sub(tmp, start = 81, end=str_locate(tmp,"dense")[,1]-8)
names(myfiles) <- labels

In [ ]:
Wang2020 <- list()
for (i in 1:length(myfiles)) {
    seu_obj <- CreateSeuratObject(myfiles[[i]], min.features = 500, min.cells = 5,
                                  project = names(myfiles[i]))
    Wang2020 <- append(Wang2020, seu_obj)
}

In [ ]:
for(i in 1:length(Wang2020)){
  cat(' #####################################\n',
      '### Processing dataset number ', i, '###\n',
      '#####################################\n')
  # add %MT
  Wang2020[[i]][["percent.MT"]]  <- PercentageFeatureSet(Wang2020[[i]], pattern = "^MT-") 
  
  # Filter out low quality cells according to the metrics defined above
  Wang2020[[i]] <- subset(Wang2020[[i]],
                           subset = percent.MT < 30 &
                             nCount_RNA > 1000
  )
  # Only mito and floor filtering; trying to find doublets
}

# preprocess each dataset individually
Wang2020 <- lapply(Wang2020, seuPreProcess)

In [ ]:
# https://github.com/mckellardw/scMuscle/blob/main/R_scripts/scMuscle_github_v1.R

bcmvn <- list()
pK <- list()
homotypic.prop <- list()
nExp_poi <- list()
nExp_poi.adj <- list()

# Estimated Doublet Rate for each dataset
edr <- estimateDoubletRate.DWM(seur.list = Wang2020)/100 #use your own known EDR here

for(i in 1:length(Wang2020)){
  cat(' ############################################\n',
      '### DoubletFinder for dataset number ', i, '###\n',
      '############################################\n')
  
  ## pK Identification (no ground-truth)
  bcmvn[[i]]<- paramSweep_v3(
    seu=Wang2020[[i]],
    PCs = 1:Wang2020[[i]]@reductions$umap_RNA@misc$n.pcs.used, 
    num.cores = 8
  ) %>% summarizeSweep(
    GT = FALSE
  ) %>% find.pK() 
  
  # Pull out max of bcmvn
  pK[[i]] <- as.numeric(as.character(bcmvn[[i]]$pK[bcmvn[[i]]$BCmetric==max(bcmvn[[i]]$BCmetric)])) # ugly, but functional...
  
  ## Homotypic Doublet Proportion Estimate
  homotypic.prop[[i]] <- modelHomotypic(Wang2020[[i]]$seurat_clusters) 
  
  nExp_poi[[i]] <- round(edr[[i]]*length(colnames(Wang2020[[i]])))  
  nExp_poi.adj[[i]] <- round(nExp_poi[[i]]*(1-homotypic.prop[[i]]))
}

In [ ]:
# Run DoubletFinder
for(i in 1:length(Wang2020)){
  Wang2020[[i]] <- 
    doubletFinder_V3.DWM_v2( # just changed it so the output metadata column name is customizable
      seu=Wang2020[[i]], 
      PCs = 1:Wang2020[[i]]@reductions$umap_RNA@misc$n.pcs.used, 
      pN = 0.25, #default value
      pK= pK[[i]], 
      nExp = nExp_poi.adj[[i]],  
      reuse.pANN = F, 
      classification.name='DF.individual', 
      pANN.name='DF.pANN.individual'
    )
}

In [ ]:
Wang2020_final <- merge(Wang2020[[1]], y = unlist(Wang2020[2:length(Wang2020)]), 
             add.cell.ids = labels, project = "Wang2020", merge.data = T)

In [ ]:
# filter doublets
Wang2020_final <- subset(Wang2020_final, subset = DF.individual == 'Singlet')

In [ ]:
# clinical metadata (created based on the Supp figure 1 -  There is no IDH mutation desribed, they are considered IDHwt)
Wang2020_clinical <- subset(data.table::fread(paste0(path.dir, 'Wang2020/clinical_data.txt')),
                            select = -c(IDH))
Wang2020_clinical$Age <- as.integer(Wang2020_clinical$Age)

In [ ]:
# no original metadata available
Wang2020_meta <- subset(data.table::fread(paste0(path.dir, 
                                                 'Wang2020/GBM_GSE139448_CellMetainfo_table.tsv')),
                          select = -c(UMAP_1,UMAP_2,Cluster))
Wang2020_meta$Cell <- gsub('@', '_X', Wang2020_meta$Cell) #change characters to macth metadata

In [ ]:
Wang2020_meta <- merge(Wang2020_meta, Wang2020_clinical, by= "Patient")
Wang2020_meta$Platform <- '10x_v2'
Wang2020_meta$Method <- 'cell'

In [ ]:
Wang2020_matrix <- GetAssayData(Wang2020_final, slot = 'counts')[,colnames(Wang2020_final) %in% Wang2020_meta$Cell]

In [ ]:
gene.names <- checkGeneSymbols(rownames(Wang2020_matrix), unmapped.as.na=FALSE)
rownames(Wang2020_matrix) <- make.unique(gene.names$Suggested.Symbol)

### Zhao2020
Microwell-seq - Raw counts matrices

In [ ]:
tidy.tables <- function(filename) {
    dat=data.table::fread(filename)
    dat <- as.data.frame(dat[,-1])
    dat <- dat[!duplicated(dat$gene), ]
    rownames(dat) <- NULL
    dat <- column_to_rownames(dat, 'gene')
    dat <- as.matrix(dat)
}

Note: Patient 031 does not exits on the metadata provided in the paper. Likely to be a typo form the authors 

In [ ]:
tmp <- list.files(path = paste0(path.dir, "Zhao2020/biopsy"),
               pattern = "*.txt.gz", 
               full.names = T)
myfiles = lapply(tmp, tidy.tables)

In [ ]:
labels <- str_sub(tmp, start = 99, end=str_locate(tmp,"cts")[,1]-2)
labels[1] <- 'PW032-702' # changed based on the manuscript and the metadata provided in GEO (see more comments below)
names(myfiles) <- labels

In [ ]:
Zhao2020 <- list()
for (i in 1:length(myfiles)) {
    seu_obj <- CreateSeuratObject(myfiles[[i]], min.features = 500, min.cells = 5, 
                                  project = names(myfiles[i]))
    Zhao2020 <- append(Zhao2020, seu_obj)
}

In [ ]:
Zhao2020_final <- merge(Zhao2020[[1]], y = unlist(Zhao2020[2:length(Zhao2020)]), 
             add.cell.ids = labels, project = "Zhao2020", merge.data = T)

In [ ]:
Zhao2020_final <- PercentageFeatureSet(Zhao2020_final, pattern = "^MT-", col.name = "percent.MT")
Zhao2020_final <- subset(Zhao2020_final, subset = percent.MT < 30 & nCount_RNA > 1000)

In [ ]:
Zhao2020_meta <- subset(data.table::fread(paste0(path.dir, 
                                                 'Zhao2020/GBM_GSE148842_CellMetainfo_table.tsv')),
                        select = -c(UMAP_1,UMAP_2,Cluster))
Zhao2020_meta$Cell <- gsub('@', '_', Zhao2020_meta$Cell) #change characters to macth cells
Zhao2020_meta <- Zhao2020_meta %>% filter(Sample == 'PW031-712' |
                                         Sample == 'PW032-701' |
                                         Sample == 'PW032-712') # this are the samples that are primary biopsies 
#this info is based in the data associated with each file in GEO. Sample PW031 has the same metadata and based in the paper
#only PW032 was profiled directly after biopsy
# tissue: glioma surgical biopsy
# age: 61
# gender: M
# location: left frontal
# diagnosis: Glioblastoma, WHO Grade IV
# treatment: none
Zhao2020_meta$Cell <- gsub('PW031-712', 'PW032-702', Zhao2020_meta$Cell) #change characters to macth cells
Zhao2020_meta$Sample <- gsub('PW031-712', 'PW032-702', Zhao2020_meta$Sample) #change characters to macth cells
Zhao2020_meta$Patient <- gsub('PW031', 'PW032', Zhao2020_meta$Sample) #change characters to macth cells

In [ ]:
# adding clinical data 
Zhao2020_meta$Age <- 61
Zhao2020_meta$Sex <- 'M'
Zhao2020_meta$Location <- 'Left frontal'
Zhao2020_meta$EGFR <- 'amplified'
Zhao2020_meta$Platform <- 'Microwell-seq'
Zhao2020_meta$Method <- 'cell'

In [ ]:
Zhao2020_matrix <- GetAssayData(Zhao2020_final, slot = 'counts')[,colnames(Zhao2020_final) %in% Zhao2020_meta$Cell]

In [ ]:
gene.names <- checkGeneSymbols(rownames(Zhao2020_matrix), unmapped.as.na=FALSE)
rownames(Zhao2020_matrix) <- make.unique(gene.names$Suggested.Symbol)

### Bhaduri2020
10x data was re-mapped (count matrices) and Fluidigm data was left out (only 400 cells)

10x matrices (downloaded BAM files, converted them to FASTQ and mapped them using CellRanger v4.0) <br>
There were less cells detected than the ones reported in the original study (they used CellRanger v2.0)

In [ ]:
tmp <- list.files(path = paste0(path.dir, "Bhaduri2020/matrices"), pattern = "^SF",
               full.names = T)
# used the raw matrices instead of the filtered (when using the filtered ones, there were only ~10K cells that matched the metadata)

In [ ]:
labels <- str_sub(tmp, start = 93) 
names(tmp) <- labels

In [ ]:
Bhaduri2020 <- list()
for (i in 1:length(tmp)) {
    seu_obj.data <- Read10X(data.dir = tmp[[i]])
    seu_obj <- CreateSeuratObject(seu_obj.data, min.features = 500, min.cells = 5, 
                                  project = names(tmp[i]))
    Bhaduri2020 <- append(Bhaduri2020, seu_obj)
}

In [ ]:
for(i in 1:length(Bhaduri2020)){
  cat(' #####################################\n',
      '### Processing dataset number ', i, '###\n',
      '#####################################\n')
  # add %MT
  Bhaduri2020[[i]][["percent.MT"]]  <- PercentageFeatureSet(Bhaduri2020[[i]], pattern = "^MT-") 
  
  # Filter out low quality cells according to the metrics defined above
  Bhaduri2020[[i]] <- subset(Bhaduri2020[[i]],
                           subset = percent.MT < 30 &
                             nCount_RNA > 1000
  )
  # Only mito and floor filtering; trying to find doublets
}

# preprocess each dataset individually
Bhaduri2020 <- lapply(Bhaduri2020, seuPreProcess)

In [ ]:
# https://github.com/mckellardw/scMuscle/blob/main/R_scripts/scMuscle_github_v1.R

bcmvn <- list()
pK <- list()
homotypic.prop <- list()
nExp_poi <- list()
nExp_poi.adj <- list()

# Estimated Doublet Rate for each dataset
edr <- estimateDoubletRate.DWM(seur.list = Bhaduri2020)/100 #use your own known EDR here

for(i in 1:length(Bhaduri2020)){
  cat(' ############################################\n',
      '### DoubletFinder for dataset number ', i, '###\n',
      '############################################\n')
  
  ## pK Identification (no ground-truth)
  bcmvn[[i]]<- paramSweep_v3(
    seu=Bhaduri2020[[i]],
    PCs = 1:Bhaduri2020[[i]]@reductions$umap_RNA@misc$n.pcs.used, 
    num.cores = 8
  ) %>% summarizeSweep(
    GT = FALSE
  ) %>% find.pK() 
  
  # Pull out max of bcmvn
  pK[[i]] <- as.numeric(as.character(bcmvn[[i]]$pK[bcmvn[[i]]$BCmetric==max(bcmvn[[i]]$BCmetric)])) # ugly, but functional...
  
  ## Homotypic Doublet Proportion Estimate
  homotypic.prop[[i]] <- modelHomotypic(Bhaduri2020[[i]]$seurat_clusters) 
  
  nExp_poi[[i]] <- round(edr[[i]]*length(colnames(Bhaduri2020[[i]])))  
  nExp_poi.adj[[i]] <- round(nExp_poi[[i]]*(1-homotypic.prop[[i]]))
}

In [ ]:
# Run DoubletFinder
for(i in 1:length(Bhaduri2020)){
  Bhaduri2020[[i]] <- 
    doubletFinder_V3.DWM_v2( # just changed it so the output metadata column name is customizable
      seu=Bhaduri2020[[i]], 
      PCs = 1:Bhaduri2020[[i]]@reductions$umap_RNA@misc$n.pcs.used, 
      pN = 0.25, #default value
      pK= pK[[i]], 
      nExp = nExp_poi.adj[[i]],  
      reuse.pANN = F, 
      classification.name='DF.individual', 
      pANN.name='DF.pANN.individual'
    )
}

In [ ]:
Bhaduri2020_final <- merge(Bhaduri2020[[1]], y = unlist(Bhaduri2020[2:length(Bhaduri2020)]), 
             add.cell.ids = labels, project = "Bhaduri2020", merge.data = T)

In [ ]:
# filter doublets
Bhaduri2020_final <- subset(Bhaduri2020_final, subset = DF.individual == 'Singlet')

In [ ]:
# clinical metadata (created based on the Supp figure 1)
Bhaduri2020_clinical <- data.table::fread(paste0(path.dir, 'Bhaduri2020/clinical_data.txt'))

In [ ]:
Bhaduri2020_meta <- subset(data.table::fread(paste0(path.dir, 'Bhaduri2020/Cell_metadata.csv')),
                           select = -c(Cluster))
Bhaduri2020_meta <- Bhaduri2020_meta %>% filter(!`Tumor ID` == 'SF11285') # this sample is Anaplastic astrocytoma (and on the paper only makes the clusters OPC, B cells)
colnames(Bhaduri2020_meta) <- c('Cell', 'Patient', 'Celltype (original)')
namevector <- c("Celltype (malignancy)", "Celltype (major-lineage)", "Celltype (minor-lineage)")
Bhaduri2020_meta[ , namevector] <- NA
Bhaduri2020_meta <- Bhaduri2020_meta[,c("Cell",
                                        "Celltype (malignancy)", 
                                        "Celltype (major-lineage)", 
                                        "Celltype (minor-lineage)",
                                       "Celltype (original)",
                                       "Patient")]
Bhaduri2020_meta$Patient <- gsub('TQ', 'SF11247', Bhaduri2020_meta$Patient) #TQ was not found in the clinical metadata
# based on the number of cells, it matches SF11247

In [ ]:
# formating cell names in metadata to match the names in matrices
SF11159 <- Bhaduri2020_meta %>% filter(Patient == 'SF11159')
SF11159 <- SF11159 %>% mutate(Cell = paste0('SF11159_', 
                                                str_sub(SF11159$Cell, start = 0, end=str_locate(SF11159$Cell,"_")[,1]-1),'-1'))
SF11209 <- Bhaduri2020_meta %>% filter(Patient == 'SF11209')
SF11209 <- SF11209 %>% mutate(Cell = paste0('SF11209_', 
                                                str_sub(SF11209$Cell, start = 0, end=str_locate(SF11209$Cell,"_")[,1]-1),'-1'))
SF11215 <- Bhaduri2020_meta %>% filter(Patient == 'SF11215')
SF11215 <- SF11215 %>% mutate(Cell = paste0('SF11215_', 
                                                str_sub(SF11215$Cell, start = 0, end=str_locate(SF11215$Cell,"SF")[,1]-1),'-1'))
SF11232 <- Bhaduri2020_meta %>% filter(Patient == 'SF11232')
SF11232 <- SF11232 %>% mutate(Cell = paste0('SF11232_', 
                                                str_sub(SF11232$Cell, start = 0, end=str_locate(SF11232$Cell,"SF")[,1]-1),'-1'))
SF11247 <- Bhaduri2020_meta %>% filter(Patient == 'SF11247')
SF11247 <- SF11247 %>% mutate(Cell = paste0('SF11247_', 
                                                str_sub(SF11247$Cell, start = 0, end=str_locate(SF11247$Cell,"_")[,1]-1),'-1'))

In [ ]:
Bhaduri2020_meta <- rbind(SF11159, SF11209, SF11215, SF11232, SF11247)
Bhaduri2020_meta <- merge(Bhaduri2020_meta, Bhaduri2020_clinical, by= "Patient")
Bhaduri2020_meta$Method <- 'cell'

In [ ]:
# to include the other good quality cells even they do not have annotated cell type metadata
cell <-  data.frame(Cell = colnames(Bhaduri2020_final))
patients <- str_sub(cell$Cell, start = 0, end=str_locate(cell$Cell,"_")[,1]-1)
meta_tmp <- cbind(cell, data.frame(Patient = patients, Method = 'cell'))
meta_tmp <- merge(meta_tmp, Bhaduri2020_clinical, by= "Patient")

In [ ]:
Bhaduri2020_meta <- list(Bhaduri2020_meta, meta_tmp) %>% reduce(full_join)

In [ ]:
Bhaduri2020_matrix <- GetAssayData(Bhaduri2020_final, 
                                     slot = 'counts')[,colnames(Bhaduri2020_final) %in% 
                                                      Bhaduri2020_meta$Cell]

In [ ]:
gene.names <- checkGeneSymbols(rownames(Bhaduri2020_matrix), unmapped.as.na=FALSE)
rownames(Bhaduri2020_matrix) <- make.unique(gene.names$Suggested.Symbol)

### Yu2020
Single-cell tagged reverse transcription–seq protocol (mentioned in Pathway-based classification of glioblastoma uncovers a mitochondrial subtype with therapeutic vulnerabilities) <br>
Data seems TPM

In [ ]:
Yu2020 <- data.table::fread(paste0(path.dir, 'Yu2020/GSE117891_all_6148.umi.count.matrix.tsv.gz')) %>% column_to_rownames('V1')

In [ ]:
Yu2020_final <- CreateSeuratObject(Yu2020, min.features = 500, min.cells = 5, 
                                  project = 'Yu2020')
Yu2020_final@assays$RNA@counts <- round(GetAssayData(Yu2020_final, slot = 'counts'))

In [ ]:
Yu2020_final <- PercentageFeatureSet(Yu2020_final, pattern = "^MT-", col.name = "percent.MT")
Yu2020_final <- subset(Yu2020_final, subset = percent.MT < 30 & nCount_RNA > 1000)

In [ ]:
Yu2020_clinical <- readxl::read_excel(paste0(path.dir, 'Yu2020/clinical_info.xlsx'), sheet = "Clinical_Info")
Yu2020_clinical$Patient <- gsub('CGGA_G', '', Yu2020_clinical$Patient) # to match with next metadata

In [ ]:
Yu2020_region <- readxl::read_excel(paste0(path.dir, 'Yu2020/clinical_info.xlsx'), sheet = "Point and Cell information")
Yu2020_region$`Sample point` <- gsub('G', '', Yu2020_region$`Sample point`) # to match with next metadata
colnames(Yu2020_region) <- c('Region', 'Tissue') # Tissue refers to the location (core, edge). Following Daranais2017 
Yu2020_region$Tissue <- gsub('Tumoral', 'Tumor', Yu2020_region$Tissue)
Yu2020_region$Tissue <- gsub('Peritumoral', 'Periphery', Yu2020_region$Tissue) # to follow the same names used by Darmanis2017

In [ ]:
Yu2020_meta <- subset(as.data.frame(readxl::read_excel(paste0(path.dir, 'Yu2020/GSE117891_Sample_barcode_cell_information.xlsx'), 
                                                sheet = "Cell_information")), select = -c(`sample_name(Library)`, bio_sample, barcode))
colnames(Yu2020_meta) <- c('Cell', 'Patient', 'Region')
Yu2020_meta <- Yu2020_meta %>% filter(!Patient == 'S4' & # gliosarcoma
                                      !Patient == 'S7' & # GBM IDHmut
                                     !Patient == 'S8' & # OA
                                     !Patient == 'S9' & # OA
                                     !Patient == 'S14') # O

In [ ]:
Yu2020_meta <- Yu2020_meta %>% merge(Yu2020_region, by = "Region") %>% merge(Yu2020_clinical, by = "Patient") %>% mutate(Platform = 'STRT-seq', Method = 'cell')
# method was infered by the citations they made to other papers but the info is nowhere in the website/paper

In [ ]:
Yu2020_matrix <- GetAssayData(Yu2020_final, slot = 'counts')[,colnames(Yu2020_final) %in% Yu2020_meta$Cell]

In [ ]:
gene.names <- checkGeneSymbols(rownames(Yu2020_matrix), unmapped.as.na=FALSE)
rownames(Yu2020_matrix) <- make.unique(gene.names$Suggested.Symbol)

### Wu2020
BD Rhapsody -  Raw count matrices

In [ ]:
tidy.tables <- function(filename) {
    dat=data.table::fread(filename) %>% t()
    colnames(dat) <- dat[1,]
    dat <- dat[-1,]
}

In [ ]:
tmp <- list.files(path = paste0(path.dir, "Wu2020/"),
               pattern = "*.csv.gz", 
               full.names = T)
myfiles = lapply(tmp, tidy.tables)

In [ ]:
labels <- str_sub(tmp, start = 80, end=str_locate(tmp,"_Mols")[,1]-1)
names(myfiles) <- labels

In [ ]:
Wu2020 <- list()
for (i in 1:length(myfiles)) {
    seu_obj <- CreateSeuratObject(myfiles[[i]], min.features = 500, min.cells = 5, 
                                  project = names(myfiles[i]))
    Wu2020 <- append(Wu2020, seu_obj)
}

In [ ]:
Wu2020_final <- merge(Wu2020[[1]], y = unlist(Wu2020[2:length(Wu2020)]), 
             add.cell.ids = labels, project = "Wu2020", merge.data = T)

In [ ]:
Wu2020_final <- PercentageFeatureSet(Wu2020_final, pattern = "^MT-", col.name = "percent.MT")
Wu2020_final <- subset(Wu2020_final, subset = percent.MT < 30 & nCount_RNA > 1000)

In [ ]:
Wu2020_clinical <- readxl::read_excel(paste0(path.dir, 'Wu2020/clinical_data.xlsx'))

In [ ]:
# creating cell metadata (is not available in the paper)
Wu2020_meta <- data.frame(Cell = colnames(Wu2020_final), 
           `Celltype (malignancy)` = rep('NA', length(colnames(Wu2020_final))),
           `Celltype (major-lineage)` = rep('NA', length(colnames(Wu2020_final))),
           `Celltype (minor-lineage)` = rep('NA', length(colnames(Wu2020_final))),
           `Celltype (original)` = rep('NA', length(colnames(Wu2020_final))),
           Patient = str_sub(colnames(Wu2020_final), start = 0, end=str_locate(colnames(Wu2020_final),"-")[,1]-1),
           Sample = str_sub(colnames(Wu2020_final), start = 0, end=str_locate(colnames(Wu2020_final),"_")[,1]-1),
           check.names = FALSE
          )

In [ ]:
Wu2020_meta <- merge(Wu2020_meta, Wu2020_clinical, by= "Sample") %>% select(-Patient.y) 
colnames(Wu2020_meta)[colnames(Wu2020_meta) == 'Patient.x'] <- 'Patient'

In [ ]:
Wu2020_matrix <- GetAssayData(Wu2020_final, slot = 'counts')[,colnames(Wu2020_final) %in% Wu2020_meta$Cell]

In [ ]:
gene.names <- checkGeneSymbols(rownames(Wu2020_matrix), unmapped.as.na=FALSE)
rownames(Wu2020_matrix) <- make.unique(gene.names$Suggested.Symbol)

### Couturier2020
10x matrices (downloaded EGA BAM files, converted them to FASTQ and mapped them using CellRanger v3.1)
10x Genomics - Raw count matrices

In [ ]:
tmp <- list.dirs(path = paste0(path.dir, "Couturier2020/cellranger_output"), recursive = FALSE)
head(tmp)

In [ ]:
labels <-  str_sub(tmp, start = 104)
names(tmp) <- labels

In [ ]:
Couturier2020 <- list()
for (i in 1:length(tmp)) {
    seu_obj.data <- Read10X(data.dir = paste0(tmp[[i]], '/outs/filtered_feature_bc_matrix'))
    seu_obj <- CreateSeuratObject(seu_obj.data, min.features = 500, min.cells = 5, 
                                  project = names(tmp[i]))
    Couturier2020 <- append(Couturier2020, seu_obj)
}

In [ ]:
for(i in 1:length(Couturier2020)){
  cat(' #####################################\n',
      '### Processing dataset number ', i, '###\n',
      '#####################################\n')
  # add %MT
  Couturier2020[[i]][["percent.MT"]]  <- PercentageFeatureSet(Couturier2020[[i]], pattern = "^MT-") 
  
  # Filter out low quality cells according to the metrics defined above
  Couturier2020[[i]] <- subset(Couturier2020[[i]],
                           subset = percent.MT < 30 &
                             nCount_RNA > 1000
  )
  # Only mito and floor filtering; trying to find doublets
}

# preprocess each dataset individually
Couturier2020 <- lapply(Couturier2020, seuPreProcess)

In [ ]:
# https://github.com/mckellardw/scMuscle/blob/main/R_scripts/scMuscle_github_v1.R

bcmvn <- list()
pK <- list()
homotypic.prop <- list()
nExp_poi <- list()
nExp_poi.adj <- list()

# Estimated Doublet Rate for each dataset
edr <- estimateDoubletRate.DWM(seur.list = Couturier2020)/100 #use your own known EDR here

for(i in 1:length(Couturier2020)){
  cat(' ############################################\n',
      '### DoubletFinder for dataset number ', i, '###\n',
      '############################################\n')
  
  ## pK Identification (no ground-truth)
  bcmvn[[i]]<- paramSweep_v3(
    seu=Couturier2020[[i]],
    PCs = 1:Couturier2020[[i]]@reductions$umap_RNA@misc$n.pcs.used, 
    num.cores = 8
  ) %>% summarizeSweep(
    GT = FALSE
  ) %>% find.pK() 
  
  # Pull out max of bcmvn
  pK[[i]] <- as.numeric(as.character(bcmvn[[i]]$pK[bcmvn[[i]]$BCmetric==max(bcmvn[[i]]$BCmetric)])) # ugly, but functional...
  
  ## Homotypic Doublet Proportion Estimate
  homotypic.prop[[i]] <- modelHomotypic(Couturier2020[[i]]$seurat_clusters) 
  
  nExp_poi[[i]] <- round(edr[[i]]*length(colnames(Couturier2020[[i]])))  
  nExp_poi.adj[[i]] <- round(nExp_poi[[i]]*(1-homotypic.prop[[i]]))
}

In [ ]:
# Run DoubletFinder
for(i in 1:length(Couturier2020)){
  Couturier2020[[i]] <- 
    doubletFinder_V3.DWM_v2( # just changed it so the output metadata column name is customizable
      seu=Couturier2020[[i]], 
      PCs = 1:Couturier2020[[i]]@reductions$umap_RNA@misc$n.pcs.used, 
      pN = 0.25, #default value
      pK= pK[[i]], 
      nExp = nExp_poi.adj[[i]],  
      reuse.pANN = F, 
      classification.name='DF.individual', 
      pANN.name='DF.pANN.individual'
    )
}

In [ ]:
Couturier2020_final <- merge(Couturier2020[[1]], y = unlist(Couturier2020[2:length(Couturier2020)]), 
             add.cell.ids = labels, project = "Couturier2020", merge.data = T)

In [ ]:
# filter doublets
Couturier2020_final <- subset(Couturier2020_final, subset = DF.individual == 'Singlet')

In [ ]:
Couturier2020_clinical <- readxl::read_excel(paste0(path.dir, 'Couturier2020/clinical_data.xlsx'))

In [ ]:
# creating cell metadata (is not available in the paper)
Couturier2020_meta <- data.frame(Cell = colnames(Couturier2020_final), 
           `Celltype (malignancy)` = rep('NA', length(colnames(Couturier2020_final))),
           `Celltype (major-lineage)` = rep('NA', length(colnames(Couturier2020_final))),
           `Celltype (minor-lineage)` = rep('NA', length(colnames(Couturier2020_final))),
           `Celltype (original)` = rep('NA', length(colnames(Couturier2020_final))),
           Patient = str_sub(colnames(Couturier2020_final), start = 0, end=str_locate(colnames(Couturier2020_final),"_")[,1]-1),
           check.names = FALSE
          )

In [ ]:
Couturier2020_meta <- merge(Couturier2020_meta, Couturier2020_clinical, by= "Patient")

In [ ]:
Couturier2020_matrix <- GetAssayData(Couturier2020_final, slot = 'counts')[,colnames(Couturier2020_final) %in% Couturier2020_meta$Cell]

In [ ]:
gene.names <- checkGeneSymbols(rownames(Couturier2020_matrix), unmapped.as.na=FALSE)
rownames(Couturier2020_matrix) <- make.unique(gene.names$Suggested.Symbol)

### Johnson2020
10x Genomics - Raw count matrices of unfiltered cells - Obtained previous formal publication

In [ ]:
tmp <- list.files(path = paste0(path.dir, "Johnson2020"), pattern = "^SM",
               full.names = T)

In [ ]:
labels <- str_sub(tmp, start = 84) 
names(tmp) <- labels

In [ ]:
Johnson2020 <- list()
for (i in 1:length(tmp)) {
    seu_obj.data <- Read10X_h5(paste0(tmp[i],'/raw_feature_bc_matrix.h5'))
    seu_obj <- CreateSeuratObject(seu_obj.data, min.features = 500, min.cells = 5, 
                                  project = names(tmp[i]))
    Johnson2020 <- append(Johnson2020, seu_obj)
}

In [ ]:
for(i in 1:length(Johnson2020)){
  cat(' #####################################\n',
      '### Processing dataset number ', i, '###\n',
      '#####################################\n')
  # add %MT
  Johnson2020[[i]][["percent.MT"]]  <- PercentageFeatureSet(Johnson2020[[i]], pattern = "^MT-") 
  
  # Filter out low quality cells according to the metrics defined above
  Johnson2020[[i]] <- subset(Johnson2020[[i]],
                           subset = percent.MT < 30 &
                             nCount_RNA > 1000
  )
  # Only mito and floor filtering; trying to find doublets
}

# preprocess each dataset individually
Johnson2020 <- lapply(Johnson2020, seuPreProcess)

In [ ]:
# https://github.com/mckellardw/scMuscle/blob/main/R_scripts/scMuscle_github_v1.R

bcmvn <- list()
pK <- list()
homotypic.prop <- list()
nExp_poi <- list()
nExp_poi.adj <- list()

# Estimated Doublet Rate for each dataset
edr <- estimateDoubletRate.DWM(seur.list = Johnson2020)/100 #use your own known EDR here

for(i in 1:length(Johnson2020)){
  cat(' ############################################\n',
      '### DoubletFinder for dataset number ', i, '###\n',
      '############################################\n')
  
  ## pK Identification (no ground-truth)
  bcmvn[[i]]<- paramSweep_v3(
    seu=Johnson2020[[i]],
    PCs = 1:Johnson2020[[i]]@reductions$umap_RNA@misc$n.pcs.used, 
    num.cores = 8
  ) %>% summarizeSweep(
    GT = FALSE
  ) %>% find.pK() 
  
  # Pull out max of bcmvn
  pK[[i]] <- as.numeric(as.character(bcmvn[[i]]$pK[bcmvn[[i]]$BCmetric==max(bcmvn[[i]]$BCmetric)])) # ugly, but functional...
  
  ## Homotypic Doublet Proportion Estimate
  homotypic.prop[[i]] <- modelHomotypic(Johnson2020[[i]]$seurat_clusters) 
  
  nExp_poi[[i]] <- round(edr[[i]]*length(colnames(Johnson2020[[i]])))  
  nExp_poi.adj[[i]] <- round(nExp_poi[[i]]*(1-homotypic.prop[[i]]))
}

In [ ]:
# Run DoubletFinder
for(i in 1:length(Johnson2020)){
  Johnson2020[[i]] <- 
    doubletFinder_V3.DWM_v2( # just changed it so the output metadata column name is customizable
      seu=Johnson2020[[i]], 
      PCs = 1:Johnson2020[[i]]@reductions$umap_RNA@misc$n.pcs.used, 
      pN = 0.25, #default value
      pK= pK[[i]], 
      nExp = nExp_poi.adj[[i]],  
      reuse.pANN = F, 
      classification.name='DF.individual', 
      pANN.name='DF.pANN.individual'
    )
}

In [ ]:
Johnson2020_final <- merge(Johnson2020[[1]], y = unlist(Johnson2020[2:length(Johnson2020)]), 
             add.cell.ids = labels, project = "Johnson2020", merge.data = T)

In [ ]:
# filter doublets
Johnson2020_final <- subset(Johnson2020_final, subset = DF.individual == 'Singlet')

In [ ]:
Johnson2020_clinical <- as.data.frame(readxl::read_excel(paste0(path.dir, 'Johnson2020/media-2.xlsx'), sheet = 'Table S1'))
Johnson2020_clinical <- Johnson2020_clinical %>% select(c(Patient, Age, Sex, Location, Stage, 
                                                          Verhaak_classification, EGFR, PTEN, p53, TERT))

In [ ]:
Johnson2020_meta <- as.data.frame(data.table::fread(paste0(path.dir, 'Johnson2020/johnson_idhwt_cell_states.txt')))
Johnson2020_meta$Cell <- paste0(Johnson2020_meta$sample_id, '_', Johnson2020_meta$cell_name)
Johnson2020_meta <- Johnson2020_meta[,-1]
colnames(Johnson2020_meta) <- c('Patient', 'Celltype (original)', 'Cell')

In [ ]:
Johnson2020_meta <- merge(Johnson2020_meta, Johnson2020_clinical, by= "Patient")

In [ ]:
# to include the other good quality cells even they do not have annotated cell type metadata
cell <-  data.frame(Cell = colnames(Johnson2020_final))
patients <- str_sub(cell$Cell, start = 0, end=str_locate(cell$Cell,"_")[,1]-1)
meta_tmp <- cbind(cell, data.frame(Patient = patients))
meta_tmp <- merge(meta_tmp, Johnson2020_clinical, by= "Patient")

In [ ]:
Johnson2020_meta <- list(Johnson2020_meta, meta_tmp) %>% reduce(full_join) %>%
        mutate(Method = 'cell', Platform = '10x_v3')

In [ ]:
Johnson2020_matrix <- GetAssayData(Johnson2020_final, slot = 'counts')[,colnames(Johnson2020_final) %in% Johnson2020_meta$Cell]

In [ ]:
gene.names <- checkGeneSymbols(rownames(Johnson2020_matrix), unmapped.as.na=FALSE)
rownames(Johnson2020_matrix) <- make.unique(gene.names$Suggested.Symbol)

### Richards2021
10x Genomics - Raw count matrices

In [ ]:
tidy.tables <- function(filename) {
    dat=data.table::fread(filename)
    dat <- as.data.frame(dat) %>% column_to_rownames('V1')
}

In [ ]:
tmp <- list.files(path = paste0(path.dir, "Richards2021/"),
               pattern = "*counts.csv.gz", 
               full.names = T)
myfiles = lapply(tmp, tidy.tables)

In [ ]:
labels <- str_sub(tmp, start = 112, end=str_locate(tmp,"_counts")[,1]-1)
names(myfiles) <- labels

In [ ]:
merged_Richards2021 <- list()
for (i in 1:length(myfiles)) {
    seu_obj <- CreateSeuratObject(myfiles[[i]], min.features = 500, min.cells = 5, 
                                  project = names(myfiles[i]))
    merged_Richards2021 <- append(merged_Richards2021, seu_obj)
}
merged_Richards2021 <- merge(merged_Richards2021[[1]], y = unlist(merged_Richards2021[2:length(merged_Richards2021)]))

In [ ]:
# matrices contain different samples. First, they are split by orig.ident
seu.list <- list()
for(i in 1:length(table(merged_Richards2021@meta.data$orig.ident))){
    seu_obj <- subset(merged_Richards2021, subset = orig.ident == names(table(merged_Richards2021@meta.data$orig.ident))[[i]])
    seu.list <- append(seu.list, seu_obj)
    }

In [ ]:
for(i in 1:length(seu.list)){
  cat(' #####################################\n',
      '### Processing dataset number ', i, '###\n',
      '#####################################\n')
  # add %MT
  seu.list[[i]][["percent.MT"]]  <- PercentageFeatureSet(seu.list[[i]], pattern = "^MT-") 
  
  # Filter out low quality cells according to the metrics defined above
  seu.list[[i]] <- subset(seu.list[[i]],
                           subset = percent.MT < 30 &
                             nCount_RNA > 1000
  )
  # Only mito and floor filtering; trying to find doublets
}

# preprocess each dataset individually
seu.list <- lapply(seu.list, seuPreProcess)

In [ ]:
# https://github.com/mckellardw/scMuscle/blob/main/R_scripts/scMuscle_github_v1.R

bcmvn <- list()
pK <- list()
homotypic.prop <- list()
nExp_poi <- list()
nExp_poi.adj <- list()

# Estimated Doublet Rate for each dataset
edr <- estimateDoubletRate.DWM(seur.list = seu.list)/100 #use your own known EDR here

for(i in 1:length(seu.list)){
  cat(' ############################################\n',
      '### DoubletFinder for dataset number ', i, '###\n',
      '############################################\n')
  
  ## pK Identification (no ground-truth)
  bcmvn[[i]]<- paramSweep_v3(
    seu=seu.list[[i]],
    PCs = 1:seu.list[[i]]@reductions$umap_RNA@misc$n.pcs.used, 
    num.cores = 8
  ) %>% summarizeSweep(
    GT = FALSE
  ) %>% find.pK() 
  
  # Pull out max of bcmvn
  pK[[i]] <- as.numeric(as.character(bcmvn[[i]]$pK[bcmvn[[i]]$BCmetric==max(bcmvn[[i]]$BCmetric)])) # ugly, but functional...
  
  ## Homotypic Doublet Proportion Estimate
  homotypic.prop[[i]] <- modelHomotypic(seu.list[[i]]$seurat_clusters) 
  
  nExp_poi[[i]] <- round(edr[[i]]*length(colnames(seu.list[[i]])))  
  nExp_poi.adj[[i]] <- round(nExp_poi[[i]]*(1-homotypic.prop[[i]]))
}

In [ ]:
# Run DoubletFinder
for(i in 1:length(seu.list)){
  seu.list[[i]] <- 
    doubletFinder_V3.DWM_v2( # just changed it so the output metadata column name is customizable
      seu=seu.list[[i]], 
      PCs = 1:seu.list[[i]]@reductions$umap_RNA@misc$n.pcs.used, 
      pN = 0.25, #default value
      pK= pK[[i]], 
      nExp = nExp_poi.adj[[i]],  
      reuse.pANN = F, 
      classification.name='DF.individual', 
      pANN.name='DF.pANN.individual'
    )
}

In [ ]:
Richards2021_final <- merge(seu.list[[1]], y = unlist(seu.list[2:length(seu.list)]), 
             project = "Richards2021", merge.data = T) # it is not necessary to add cell IDs here because they are already in the name of each cell

In [ ]:
# filter doublets
Richards2021_final <- subset(Richards2021_final, subset = DF.individual == 'Singlet')

In [ ]:
Richards2021_clinical <- subset(as.data.frame(readxl::read_excel(paste0(path.dir, 'Richards2021/43018_2020_154_MOESM3_ESM.xlsx'))), 
                               select = c(PatientID, Age, Sex)) %>% filter(!PatientID == 'G620') # this is an IDH mutant sample
colnames(Richards2021_clinical)[1] <- 'Patient'
Richards2021_clinical$Sex <- gsub('Male', 'M', Richards2021_clinical$Sex)
Richards2021_clinical$Sex <- gsub('Female', 'M', Richards2021_clinical$Sex)

In [ ]:
scRNA_Richards2021 <- as.data.frame(data.table::fread(paste0(path.dir, 
                                                             "Richards2021/Richards_NatureCancer_GBM_scRNAseq_meta.csv.gz")))
scRNA_Richards2021 <- subset(scRNA_Richards2021, select = c(V1, PatientID, Stage, CellType)) %>% mutate(Method = 'cell', Platform = '10x_v2')
scRNA_Richards2021$V1 <- gsub('-', '.', scRNA_Richards2021$V1) # change characters to match cell names
snRNA_Richards2021 <- as.data.frame(data.table::fread(paste0(path.dir, 
                                                             "Richards2021/Richards_NatureCancer_GBM_snRNAseq_meta.csv.gz")))
snRNA_Richards2021 <- subset(snRNA_Richards2021, select = c(V1, PatientID, Stage)) %>% mutate(Method = 'nuclei', Platform = '10x_v2', CellType = NA)

In [ ]:
Richards2021_meta <- rbind(scRNA_Richards2021, snRNA_Richards2021) %>% filter(!PatientID == 'G620') # this is an IDH mutant sample
colnames(Richards2021_meta) <- c('Cell', 'Patient', 'Stage', 'Celltype (original)', 'Method', 'Platform')
Richards2021_meta$Stage <- gsub('PRIMARY', 'Primary', Richards2021_meta$Stage)
Richards2021_meta$Stage <- gsub('RECURRENT', 'Recurrent', Richards2021_meta$Stage)

In [ ]:
Richards2021_meta <- merge(Richards2021_meta, Richards2021_clinical, by= "Patient")

In [ ]:
# to include the other good quality cells even they do not have annotated cell type metadata

cell <-  data.frame(Cell = colnames(Richards2021_final))
patients <- str_sub(cell$Cell, start = 0, end=str_locate(cell$Cell,"_T")[,1]-1)
meta_tmp <- cbind(cell, data.frame(Patient = patients))
meta_tmp$Patient <- gsub('\\.\\w+', '', meta_tmp$Patient) # https://cran.r-project.org/web/packages/stringr/vignettes/regular-expressions.html
meta_tmp <- meta_tmp %>% mutate(Method = 'cell', Platform = '10x_v2')

In [ ]:
Richards2021_meta <- list(Richards2021_meta, meta_tmp) %>% reduce(full_join)

In [ ]:
Richards2021_matrix <- GetAssayData(Richards2021_final, slot = 'counts')[,colnames(Richards2021_final) %in% Richards2021_meta$Cell]

In [ ]:
gene.names <- checkGeneSymbols(rownames(Richards2021_matrix), unmapped.as.na=FALSE)
rownames(Richards2021_matrix) <- make.unique(gene.names$Suggested.Symbol)

## Studies focused on immune cells

### Darmanis2017
Smart-seq2 - TPM matrices <br>
Matrices include only filtered cells by QC

In [ ]:
Darmanis2017 <- data.table::fread(paste0(path.dir, 'Darmanis2017/GSE84465_GBM_All_data.csv')) %>% column_to_rownames('V1')
n <-dim(Darmanis2017)[1]
Darmanis2017 <- Darmanis2017[1:(n-5),] # last columns where some quality metrics (# feautres, aligment, etc) embebed in the count matrix
Darmanis2017

In [ ]:
Darmanis2017_final <- CreateSeuratObject(Darmanis2017, min.features = 500, min.cells = 5, 
                                  project = 'Darmanis2017')

In [ ]:
Darmanis2017_final <- PercentageFeatureSet(Darmanis2017_final, pattern = "^MT-", col.name = "percent.MT")
Darmanis2017_final <- subset(Darmanis2017_final, subset = percent.MT < 30)

In [ ]:
# clinical metadata
Darmanis2017_clinical <- subset(data.table::fread(paste0(path.dir, 'Darmanis2017/clinical_data.txt')),
                               select = -c(IDH, `Final Path`))

In [ ]:
Darmanis2017_meta <- subset(data.table::fread(paste0(path.dir, 'Darmanis2017/GBM_GSE84465_CellMetainfo_table.tsv')),
                     select = -c(UMAP_1,UMAP_2,Cluster, Selection)) # selection means whether was enriched for CD45+, etc
# tissue in this case is region (periphery vs core)

In [ ]:
Darmanis2017_meta <- merge(Darmanis2017_meta, Darmanis2017_clinical, by= "Patient")
Darmanis2017_meta$Platform <- 'Smart-seq2'
Darmanis2017_meta$Method <- 'cell'

In [ ]:
Darmanis2017_matrix <- GetAssayData(Darmanis2017_final, 
                                     slot = 'counts')[,colnames(Darmanis2017_final) %in% 
                                                      Darmanis2017_meta$Cell]

In [ ]:
gene.names <- checkGeneSymbols(rownames(Darmanis2017_matrix), unmapped.as.na=FALSE)
rownames(Darmanis2017_matrix) <- make.unique(gene.names$Suggested.Symbol)

### Sankowski2019
CEL-seq2 - Count matrices normalized by downscaling. Roughly raw counts. It was created using RACEID. <br>
Check https://github.com/dgrun/RaceID3_StemID2_package/blob/master/R/RaceID.R line 22

In [ ]:
tidy.tables <- function(filename) {
    dat=read.table(filename)
    dat$V1 <-sub("\\_.*","\\", dat$V1)
    dat <- dat[!str_detect(dat$V1, "^[:lower:].*$"), ]
    dat <- dat[!str_detect(dat$V1, "^[:number:].*$"), ]
    colnames(dat) <- dat[1,]
    dat <- dat[-1,]
    dat <- dat[!duplicated(dat$GENEID), ]
    rownames(dat) <- NULL
    dat <- column_to_rownames(dat, 'GENEID')
    dat <- as.matrix(dat)
}

In [ ]:
tmp <- list.files(path = paste0(path.dir, "Sankowski2019"),
               pattern = "*.tsv.gz", 
               full.names = T)
myfiles = lapply(tmp, tidy.tables)

In [ ]:
labels <- str_sub(tmp, start = 97, end=str_locate(tmp,"coutt")[,1]-2) 
names(myfiles) <- labels

In [ ]:
Sankowski2019 <- list()
for (i in 1:length(myfiles)) {
    seu_obj <- CreateSeuratObject(myfiles[[i]], min.features = 0, min.cells = 5, 
                                  project = names(myfiles[i]))
    Sankowski2019 <- append(Sankowski2019, seu_obj)
}

In [ ]:
Sankowski2019_final <- merge(Sankowski2019[[1]], y = unlist(Sankowski2019[2:length(Sankowski2019)]), 
             add.cell.ids = labels, project = "Sankowski2019", merge.data = T)

In [ ]:
Sankowski2019_final <- PercentageFeatureSet(Sankowski2019_final, pattern = "^MT-", col.name = "percent.MT")
Sankowski2019_final <- subset(Sankowski2019_final, subset = percent.MT < 30 & nCount_RNA > 1000 & nFeature_RNA > 500) 

In [ ]:
# clinical metadata
Sankowski2019_clinical <- subset(data.table::fread(paste0(path.dir, 'Sankowski2019/clinical_data.txt')))

In [ ]:
Sankowski2019_meta <- subset(data.table::fread(paste0(path.dir, 
                                               'Sankowski2019/GBM_GSE135437_CellMetainfo_table.tsv')),
                             select = -c(UMAP_1,UMAP_2,Cluster))
Sankowski2019_meta <- Sankowski2019_meta[!grepl("37_filtered", Sankowski2019_meta$Cell),] # get rid of cells named 'filtered...'
Sankowski2019_meta$Cell <- gsub('@', '_', Sankowski2019_meta$Cell) # change characters to match cell names
Sankowski2019_meta <- Sankowski2019_meta %>% filter(Patient == 'Pat6' | Patient == 'Pat9' | 
                                                   Patient == 'Pat13' | Patient == 'Pat14' |
                                                   Patient == 'GBM1' | Patient == 'GBM2' |
                                                   Patient == 'GBM3' | Patient == 'GBM4'
                                                   )

In [ ]:
Sankowski2019_meta <- merge(Sankowski2019_meta, Sankowski2019_clinical, by= "Patient")
Sankowski2019_meta$Platform <- 'Cel-seq2'
Sankowski2019_meta$Method <- 'cell'

In [ ]:
Sankowski2019_matrix <- round(GetAssayData(Sankowski2019_final, # added round to get rid of the decimals
                                     slot = 'counts'))[,colnames(Sankowski2019_final) %in% 
                                                      Sankowski2019_meta$Cell]

In [ ]:
gene.names <- checkGeneSymbols(rownames(Sankowski2019_matrix), unmapped.as.na=FALSE)
rownames(Sankowski2019_matrix) <- make.unique(gene.names$Suggested.Symbol)

### Goswami2019
10x Genomics - Count matrices

In [ ]:
tidy.tables <- function(filename) {
    dat <- data.table::fread(filename)
    dat <- as.data.frame(dat[,-2])
    dat$V1 <- seq(0, length(dat$V1)-1)
    dat <- t(dat)
    colnames(dat) <- dat[1,]
    dat <- dat[-1,]
    dat <- dat[!duplicated(rownames(dat)), ] # there are ~5 genes duplicated in each table (COG8. MATR3, SPATA13, some LINC genes)
}


In [ ]:
tmp <- list.files(paste0(path.dir, "Goswami2019"),
               pattern = "*.csv", 
               full.names = T)
myfiles = lapply(tmp, tidy.tables)

In [ ]:
labels <- str_sub(tmp, start = 84, end=str_locate(tmp,"output")[,1]-2)
names(myfiles) <- labels

In [ ]:
Goswami2019 <- list()
for (i in 1:length(myfiles)) {
    seu_obj <- CreateSeuratObject(myfiles[[i]], min.features = 500, min.cells = 5, 
                                  project = names(myfiles[i]))
    Goswami2019 <- append(Goswami2019, seu_obj)
}

In [ ]:
for(i in 1:length(Goswami2019)){
  cat(' #####################################\n',
      '### Processing dataset number ', i, '###\n',
      '#####################################\n')
  # add %MT
  Goswami2019[[i]][["percent.MT"]]  <- PercentageFeatureSet(Goswami2019[[i]], pattern = "^MT-") 
  
  # Filter out low quality cells according to the metrics defined above
  Goswami2019[[i]] <- subset(Goswami2019[[i]],
                           subset = percent.MT < 30 &
                             nCount_RNA > 1000
  )
  # Only mito and floor filtering; trying to find doublets
}

# preprocess each dataset individually
Goswami2019 <- lapply(Goswami2019, seuPreProcess)

In [ ]:
# https://github.com/mckellardw/scMuscle/blob/main/R_scripts/scMuscle_github_v1.R

bcmvn <- list()
pK <- list()
homotypic.prop <- list()
nExp_poi <- list()
nExp_poi.adj <- list()

# Estimated Doublet Rate for each dataset
edr <- estimateDoubletRate.DWM(seur.list = Goswami2019)/100 #use your own known EDR here

for(i in 1:length(Goswami2019)){
  cat(' ############################################\n',
      '### DoubletFinder for dataset number ', i, '###\n',
      '############################################\n')
  
  ## pK Identification (no ground-truth)
  bcmvn[[i]]<- paramSweep_v3(
    seu=Goswami2019[[i]],
    PCs = 1:Goswami2019[[i]]@reductions$umap_RNA@misc$n.pcs.used, 
    num.cores = 8
  ) %>% summarizeSweep(
    GT = FALSE
  ) %>% find.pK() 
  
  # Pull out max of bcmvn
  pK[[i]] <- as.numeric(as.character(bcmvn[[i]]$pK[bcmvn[[i]]$BCmetric==max(bcmvn[[i]]$BCmetric)])) # ugly, but functional...
  
  ## Homotypic Doublet Proportion Estimate
  homotypic.prop[[i]] <- modelHomotypic(Goswami2019[[i]]$seurat_clusters) 
  
  nExp_poi[[i]] <- round(edr[[i]]*length(colnames(Goswami2019[[i]])))  
  nExp_poi.adj[[i]] <- round(nExp_poi[[i]]*(1-homotypic.prop[[i]]))
}

In [ ]:
# Run DoubletFinder
for(i in 1:length(Goswami2019)){
  Goswami2019[[i]] <- 
    doubletFinder_V3.DWM_v2( # just changed it so the output metadata column name is customizable
      seu=Goswami2019[[i]], 
      PCs = 1:Goswami2019[[i]]@reductions$umap_RNA@misc$n.pcs.used, 
      pN = 0.25, #default value
      pK= pK[[i]], 
      nExp = nExp_poi.adj[[i]],  
      reuse.pANN = F, 
      classification.name='DF.individual', 
      pANN.name='DF.pANN.individual'
    )
}

In [ ]:
Goswami2019_final <- merge(Goswami2019[[1]], y = unlist(Goswami2019[2:length(Goswami2019)]), 
             add.cell.ids = labels, project = "Goswami2019", merge.data = T)

In [ ]:
# filter doublets
Goswami2019_final <- subset(Goswami2019_final, subset = DF.individual == 'Singlet')

In [ ]:
Goswami2019_clinical <- subset(readxl::read_excel(paste0(path.dir, 'Goswami2019/clinical_data.xlsx')),
                               select = -c(IDH)) %>% filter(Patient != '3199') # this is a GBM IDH mutant 

In [ ]:
Goswami2019_meta <- data.frame(Cell = colnames(Goswami2019_final), 
           `Celltype (malignancy)` = rep('Immune cells', length(colnames(Goswami2019_final))), # this are CD45+ sorted cells
           `Celltype (major-lineage)` = rep('NA', length(colnames(Goswami2019_final))),
           `Celltype (minor-lineage)` = rep('NA', length(colnames(Goswami2019_final))),
           `Celltype (original)` = rep('NA', length(colnames(Goswami2019_final))),
           Patient = str_sub(colnames(Goswami2019_final), start = 0, end=str_locate(colnames(Goswami2019_final),"_")[,1]-1),
           check.names = FALSE
          )

In [ ]:
Goswami2019_meta <- merge(Goswami2019_meta, Goswami2019_clinical, by= "Patient")

In [ ]:
Goswami2019_matrix <- GetAssayData(Goswami2019_final, slot = 'counts')[,colnames(Goswami2019_final) %in% Goswami2019_meta$Cell]

In [ ]:
gene.names <- checkGeneSymbols(rownames(Goswami2019_matrix), unmapped.as.na=FALSE)
rownames(Goswami2019_matrix) <- make.unique(gene.names$Suggested.Symbol)

### Pombo2021
10x Genomics - Raw count matrices (filtered for only immune cells) - 3' and 5' capture

In [ ]:
tidy.tables <- function(filename) {
    dat=data.table::fread(filename)
    dat <- as.data.frame(dat) %>% column_to_rownames('V1')
}

In [ ]:
tmp <- list.files(path = paste0(path.dir, "Pombo2021"),
               pattern = "*matrix.csv", 
               full.names = T)
myfiles = lapply(tmp, tidy.tables)

In [ ]:
labels <- str_sub(tmp, start = 82, end=str_locate(tmp,".filtered")[,1]-1)
names(myfiles) <- labels

In [ ]:
tmp_r <- CreateSeuratObject(myfiles[[1]], min.features = 0, min.cells = 5)
tmp_nd <- CreateSeuratObject(myfiles[[2]], min.features = 0, min.cells = 5)
tmp_rnd <- CreateSeuratObject(myfiles[[3]], min.features = 0, min.cells = 5)

In [ ]:
nd_Pombo2021 <- as.data.frame(data.table::fread(paste0(path.dir, 
                                                       "Pombo2021/GSM4972211_annot.Human.GBM.ND1_2_3_4_5_6_7.csv")))
nd_Pombo2021 <- subset(nd_Pombo2021, select = c(cell, sample)) %>% column_to_rownames('cell')

r_Pombo2021 <- as.data.frame(data.table::fread(paste0(path.dir, 
                                                             "Pombo2021/GSM4972210_annot.Human.GBM.R1_2_3_4_4nc.csv")))
r_Pombo2021 <- subset(r_Pombo2021, select = c(cell, sample)) %>% column_to_rownames('cell')

rnd_Pombo2021 <- as.data.frame(data.table::fread(paste0(path.dir, 
                                                       "Pombo2021/GSM4972212_annot.Citeseq_Human.GBM.R2_5_ND8.csv")))
rnd_Pombo2021 <- subset(rnd_Pombo2021, select = c(V7, V8)) %>% `colnames<-` (c('cell', 'sample')) %>% column_to_rownames('cell') 

In [ ]:
tmp_r <- AddMetaData(tmp_r, r_Pombo2021)
tmp_nd <- AddMetaData(tmp_nd, nd_Pombo2021)
tmp_rnd <- AddMetaData(tmp_rnd, rnd_Pombo2021)

In [ ]:
# matrices contain different samples. First, they are split by sample
seu.list_r <- list()
for(i in 1:length(table(tmp_r@meta.data$sample))){
    seu_obj <- subset(tmp_r, subset = sample == names(table(tmp_r@meta.data$sample))[[i]])
    seu.list_r <- append(seu.list_r, seu_obj)
    }

seu.list_nd <- list()
for(i in 1:length(table(tmp_nd@meta.data$sample))){
    seu_obj <- subset(tmp_nd, subset = sample == names(table(tmp_nd@meta.data$sample))[[i]])
    seu.list_nd <- append(seu.list_nd, seu_obj)
    }

seu.list_rnd <- list()
for(i in 1:length(table(tmp_rnd@meta.data$sample))){
    seu_obj <- subset(tmp_rnd, subset = sample == names(table(tmp_rnd@meta.data$sample))[[i]])
    seu.list_rnd <- append(seu.list_rnd, seu_obj)
    }


In [ ]:
seu.list <- c(seu.list_r, seu.list_nd, seu.list_rnd)
for(i in 1:length(seu.list)){
  cat(' #####################################\n',
      '### Processing dataset number ', i, '###\n',
      '#####################################\n')
  # add %MT
  seu.list[[i]][["percent.MT"]]  <- PercentageFeatureSet(seu.list[[i]], pattern = "^MT-") 
  
  # Filter out low quality cells according to the metrics defined above
  seu.list[[i]] <- subset(seu.list[[i]],
                           subset = percent.MT < 30 &
                             nCount_RNA > 1000 & nFeature_RNA > 500
  )
  # Only mito and floor filtering; trying to find doublets
}

# preprocess each dataset individually
seu.list <- lapply(seu.list, seuPreProcess)

In [ ]:
# https://github.com/mckellardw/scMuscle/blob/main/R_scripts/scMuscle_github_v1.R

bcmvn <- list()
pK <- list()
homotypic.prop <- list()
nExp_poi <- list()
nExp_poi.adj <- list()

# Estimated Doublet Rate for each dataset
edr <- estimateDoubletRate.DWM(seur.list = seu.list)/100 #use your own known EDR here

for(i in 1:length(seu.list)){
  cat(' ############################################\n',
      '### DoubletFinder for dataset number ', i, '###\n',
      '############################################\n')
  
  ## pK Identification (no ground-truth)
  bcmvn[[i]]<- paramSweep_v3(
    seu=seu.list[[i]],
    PCs = 1:seu.list[[i]]@reductions$umap_RNA@misc$n.pcs.used, 
    num.cores = 8
  ) %>% summarizeSweep(
    GT = FALSE
  ) %>% find.pK() 
  
  # Pull out max of bcmvn
  pK[[i]] <- as.numeric(as.character(bcmvn[[i]]$pK[bcmvn[[i]]$BCmetric==max(bcmvn[[i]]$BCmetric)])) # ugly, but functional...
  
  ## Homotypic Doublet Proportion Estimate
  homotypic.prop[[i]] <- modelHomotypic(seu.list[[i]]$seurat_clusters) 
  
  nExp_poi[[i]] <- round(edr[[i]]*length(colnames(seu.list[[i]])))  
  nExp_poi.adj[[i]] <- round(nExp_poi[[i]]*(1-homotypic.prop[[i]]))
}

In [ ]:
# Run DoubletFinder
for(i in 1:length(seu.list)){
  seu.list[[i]] <- 
    doubletFinder_V3.DWM_v2( # just changed it so the output metadata column name is customizable
      seu=seu.list[[i]], 
      PCs = 1:seu.list[[i]]@reductions$umap_RNA@misc$n.pcs.used, 
      pN = 0.25, #default value
      pK= pK[[i]], 
      nExp = nExp_poi.adj[[i]],  
      reuse.pANN = F, 
      classification.name='DF.individual', 
      pANN.name='DF.pANN.individual'
    )
}

In [ ]:
labels <- c(names(table(r_Pombo2021)), names(table(nd_Pombo2021)), names(table(rnd_Pombo2021)))

In [ ]:
Pombo2021_final <- merge(seu.list[[1]], y = unlist(seu.list[2:length(seu.list)]), 
             add.cell.ids = labels, project = "Pombo2021", merge.data = T)

In [ ]:
# filter doublets
Pombo2021_final <- subset(Pombo2021_final, subset = DF.individual == 'Singlet')

In [ ]:
Pombo2021_clinical <- subset(as.data.frame(readxl::read_excel(paste0(path.dir, 'Pombo2021/41593_2020_789_MOESM3_ESM.xlsx'))), 
                               select = -c(`IDH status`)) %>% mutate(Method = 'cell')

In [ ]:
nd_Pombo2021 <- as.data.frame(data.table::fread(paste0(path.dir, 
                                                       "Pombo2021/GSM4972211_annot.Human.GBM.ND1_2_3_4_5_6_7.csv")))
nd_Pombo2021 <- subset(nd_Pombo2021, select = c(cell, sample, cluster)) 
r_Pombo2021 <- as.data.frame(data.table::fread(paste0(path.dir, 
                                                             "Pombo2021/GSM4972210_annot.Human.GBM.R1_2_3_4_4nc.csv")))
r_Pombo2021 <- subset(r_Pombo2021, select = c(cell, sample, cluster))
rnd_Pombo2021 <- as.data.frame(data.table::fread(paste0(path.dir, 
                                                       "Pombo2021/GSM4972212_annot.Citeseq_Human.GBM.R2_5_ND8.csv")))
rnd_Pombo2021 <- subset(rnd_Pombo2021, select = c(sample, V7, V8)) %>% `colnames<-` (c('cluster','cell', 'sample'))
Pombo2021_meta <- rbind(nd_Pombo2021, rnd_Pombo2021, r_Pombo2021)
colnames(Pombo2021_meta) <- c('Cell', 'Patient', 'Celltype (original)')
Pombo2021_meta$Cell <- paste0(Pombo2021_meta$Patient, '_', Pombo2021_meta$Cell)

In [ ]:
Pombo2021_meta <- merge(Pombo2021_meta, Pombo2021_clinical, by= "Patient")

In [ ]:
Pombo2021_matrix <- GetAssayData(Pombo2021_final, slot = 'counts')[,colnames(Pombo2021_final) %in% Pombo2021_meta$Cell]

In [ ]:
gene.names <- checkGeneSymbols(rownames(Pombo2021_matrix), unmapped.as.na=FALSE)
rownames(Pombo2021_matrix) <- make.unique(gene.names$Suggested.Symbol)

### Mathewson2021
10x Genomics - Raw count matrices - 5' capture

In [ ]:
tmp <- list.files(path = paste0(path.dir, "Mathewson2021"), pattern = "*.h5",
               full.names = T)

In [ ]:
labels <- str_sub(tmp, start = 97, end=str_locate(tmp,"_raw")[,1]-1)

In [ ]:
Mathewson2021 <- list()
for (i in 1:length(tmp)) {
    seu_obj.data <- Read10X_h5(paste0(tmp[i]))
    seu_obj <- CreateSeuratObject(seu_obj.data, min.features = 500, min.cells = 5, 
                                  project = labels[i])
    Mathewson2021 <- append(Mathewson2021, seu_obj)
}

In [ ]:
for(i in 1:length(Mathewson2021)){
  cat(' #####################################\n',
      '### Processing dataset number ', i, '###\n',
      '#####################################\n')
  # add %MT
  Mathewson2021[[i]][["percent.MT"]]  <- PercentageFeatureSet(Mathewson2021[[i]], pattern = "^MT-") 
  
  # Filter out low quality cells according to the metrics defined above
  Mathewson2021[[i]] <- subset(Mathewson2021[[i]],
                           subset = percent.MT < 30 &
                             nCount_RNA > 1000
  )
  # Only mito and floor filtering; trying to find doublets
}

# preprocess each dataset individually
Mathewson2021 <- lapply(Mathewson2021, seuPreProcess)

In [ ]:
# https://github.com/mckellardw/scMuscle/blob/main/R_scripts/scMuscle_github_v1.R

bcmvn <- list()
pK <- list()
homotypic.prop <- list()
nExp_poi <- list()
nExp_poi.adj <- list()

# Estimated Doublet Rate for each dataset
edr <- estimateDoubletRate.DWM(seur.list = Mathewson2021)/100 #use your own known EDR here

for(i in 1:length(Mathewson2021)){
  cat(' ############################################\n',
      '### DoubletFinder for dataset number ', i, '###\n',
      '############################################\n')
  
  ## pK Identification (no ground-truth)
  bcmvn[[i]]<- paramSweep_v3(
    seu=Mathewson2021[[i]],
    PCs = 1:Mathewson2021[[i]]@reductions$umap_RNA@misc$n.pcs.used, 
    num.cores = 8
  ) %>% summarizeSweep(
    GT = FALSE
  ) %>% find.pK() 
  
  # Pull out max of bcmvn
  pK[[i]] <- as.numeric(as.character(bcmvn[[i]]$pK[bcmvn[[i]]$BCmetric==max(bcmvn[[i]]$BCmetric)])) # ugly, but functional...
  
  ## Homotypic Doublet Proportion Estimate
  homotypic.prop[[i]] <- modelHomotypic(Mathewson2021[[i]]$seurat_clusters) 
  
  nExp_poi[[i]] <- round(edr[[i]]*length(colnames(Mathewson2021[[i]])))  
  nExp_poi.adj[[i]] <- round(nExp_poi[[i]]*(1-homotypic.prop[[i]]))
}

In [ ]:
# Run DoubletFinder
for(i in 1:length(Mathewson2021)){
  Mathewson2021[[i]] <- 
    doubletFinder_V3.DWM_v2( # just changed it so the output metadata column name is customizable
      seu=Mathewson2021[[i]], 
      PCs = 1:Mathewson2021[[i]]@reductions$umap_RNA@misc$n.pcs.used, 
      pN = 0.25, #default value
      pK= pK[[i]], 
      nExp = nExp_poi.adj[[i]],  
      reuse.pANN = F, 
      classification.name='DF.individual', 
      pANN.name='DF.pANN.individual'
    )
}

In [ ]:
Mathewson2021_final <- merge(Mathewson2021[[1]], y = unlist(Mathewson2021[2:length(Mathewson2021)]), 
             add.cell.ids = paste0(labels, '_GEX'), project = "Mathewson2021", merge.data = T)

In [ ]:
# filter doublets
Mathewson2021_final <- subset(Mathewson2021_final, subset = DF.individual == 'Singlet')

In [ ]:
Mathewson2021_clinical <- as.data.frame(readxl::read_excel(paste0(path.dir, 'Mathewson2021/1-s2.0-S0092867421000659-mmc1.xlsx'))) %>% 
                                        select(-c(`IDH Status`))
colnames(Mathewson2021_clinical)[4] <- 'Sex'

In [ ]:
Mathewson2021_meta <- as.data.frame(fread(paste0(path.dir, 'Mathewson2021/GSE163108_metadata_10x.csv'))) %>% select(c(V1, sampleid, annotate_Tcelltype))
colnames(Mathewson2021_meta) <- c('Cell', 'Patient','Celltype (original)')
Mathewson2021_meta$Cell <- paste0(Mathewson2021_meta$Cell, '-1')
Mathewson2021_meta$Patient <- gsub('_GEX', '', Mathewson2021_meta$Patient)

In [ ]:
Mathewson2021_meta <- merge(Mathewson2021_meta, Mathewson2021_clinical, by= "Patient")

In [ ]:
# to include the other good quality cells even they do not have annotated cell type metadata
cell <-  data.frame(Cell = colnames(Mathewson2021_final))
patients <- str_sub(cell$Cell, start = 0, end=str_locate(cell$Cell,"_")[,1]-1)
meta_tmp <- cbind(cell, data.frame(Patient = patients))
meta_tmp <- merge(meta_tmp, Mathewson2021_clinical, by= "Patient")

In [ ]:
Mathewson2021_meta <- list(Mathewson2021_meta, meta_tmp) %>% reduce(full_join) %>% mutate(Platform = "10x_v1_5'", Method = 'cell')

In [ ]:
Mathewson2021_matrix <- GetAssayData(Mathewson2021_final, slot = 'counts')[,colnames(Mathewson2021_final) %in% Mathewson2021_meta$Cell]

In [ ]:
gene.names <- checkGeneSymbols(rownames(Mathewson2021_matrix), unmapped.as.na=FALSE)
rownames(Mathewson2021_matrix) <- make.unique(gene.names$Suggested.Symbol)

## Our study

In [ ]:
path.dir <- list.dirs(path = '/hpc/pmc_stunnenberg/cruiz/scRNA/cellranger-output/gbm/deep_sequencing', recursive = FALSE)
path.dir <- path.dir[-1] # delete script dir

In [ ]:
labels <- paste0('NH', str_sub(path.dir, start = 78))
names(path.dir) <- labels

In [ ]:
gbm_cohort <- list()
for (i in 1:length(path.dir)) {
    seu_obj.data <- Read10X(data.dir = paste0(path.dir[[i]], '/outs/filtered_feature_bc_matrix'))
    seu_obj <- CreateSeuratObject(seu_obj.data, min.features = 500, min.cells = 5, 
                                  project = names(path.dir[i]))
    gbm_cohort <- append(gbm_cohort, seu_obj)
}

In [ ]:
for(i in 1:length(gbm_cohort)){
  cat(' #####################################\n',
      '### Processing dataset number ', i, '###\n',
      '#####################################\n')
  # add %MT
  gbm_cohort[[i]][["percent.MT"]]  <- PercentageFeatureSet(gbm_cohort[[i]], pattern = "^MT-") 
  
  # Filter out low quality cells according to the metrics defined above
  gbm_cohort[[i]] <- subset(gbm_cohort[[i]],
                           subset = percent.MT < 10 &
                             nCount_RNA > 1000
  )
  # Only mito and floor filtering; trying to find doublets
}

# preprocess each dataset individually
gbm_cohort <- lapply(gbm_cohort, seuPreProcess)

In [ ]:
# https://github.com/mckellardw/scMuscle/blob/main/R_scripts/scMuscle_github_v1.R

bcmvn <- list()
pK <- list()
homotypic.prop <- list()
nExp_poi <- list()
nExp_poi.adj <- list()

# Estimated Doublet Rate for each dataset
edr <- estimateDoubletRate.DWM(seur.list = gbm_cohort)/100 #use your own known EDR here

for(i in 1:length(gbm_cohort)){
  cat(' ############################################\n',
      '### DoubletFinder for dataset number ', i, '###\n',
      '############################################\n')
  
  ## pK Identification (no ground-truth)
  bcmvn[[i]]<- paramSweep_v3(
    seu=gbm_cohort[[i]],
    PCs = 1:gbm_cohort[[i]]@reductions$umap_RNA@misc$n.pcs.used, 
    num.cores = 4
  ) %>% summarizeSweep(
    GT = FALSE
  ) %>% find.pK() 
  
  # Pull out max of bcmvn
  pK[[i]] <- as.numeric(as.character(bcmvn[[i]]$pK[bcmvn[[i]]$BCmetric==max(bcmvn[[i]]$BCmetric)])) # ugly, but functional...
  
  ## Homotypic Doublet Proportion Estimate
  homotypic.prop[[i]] <- modelHomotypic(gbm_cohort[[i]]$seurat_clusters) 
  
  nExp_poi[[i]] <- round(edr[[i]]*length(colnames(gbm_cohort[[i]])))  
  nExp_poi.adj[[i]] <- round(nExp_poi[[i]]*(1-homotypic.prop[[i]]))
}

In [ ]:
# Run DoubletFinder
for(i in 1:length(gbm_cohort)){
  gbm_cohort[[i]] <- 
    doubletFinder_V3.DWM_v2( # just changed it so the output metadata column name is customizable
      seu=gbm_cohort[[i]], 
      PCs = 1:gbm_cohort[[i]]@reductions$umap_RNA@misc$n.pcs.used, 
      pN = 0.25, #default value
      pK= pK[[i]], 
      nExp = nExp_poi.adj[[i]],  
      reuse.pANN = F, 
      classification.name='DF.individual', 
      pANN.name='DF.pANN.individual'
    )
}

In [ ]:
Ruiz2021_final <- merge(gbm_cohort[[1]], y = unlist(gbm_cohort[2:length(gbm_cohort)]), 
             add.cell.ids = labels, project = "gbm_cohort", merge.data = T)

In [ ]:
# filter doublets
Ruiz2021_final <- subset(Ruiz2021_final, subset = DF.individual == 'Singlet')

In [ ]:
Ruiz2021_clinical <- subset(as.data.frame(readxl::read_excel('/hpc/pmc_stunnenberg/cruiz/scRNA/analysis/gbm/deep_sequencing/notebooks/singlets-per-sample/sample_metadata.xlsx')), 
                               select = -c(Diagnosis, `Pathology comments`)) %>% `rownames<-`(NULL)
Ruiz2021_clinical$Age <- as.numeric(Ruiz2021_clinical$Age)

In [ ]:
Ruiz2021_meta <- as.data.frame(data.table::fread('/hpc/pmc_stunnenberg/cruiz/scRNA/analysis/gbm/deep_sequencing/notebooks/cell_annotation.csv'))
Ruiz2021_meta$Cell <- gsub('NH16_', 'NH16-', Ruiz2021_meta$Cell) # change characters to match cell names
Ruiz2021_meta$Patient <- gsub('NH16_', 'NH16-', Ruiz2021_meta$Patient) # change characters to match cell names
Ruiz2021_meta$Cell <- gsub('NH17_', 'NH17-', Ruiz2021_meta$Cell) # change characters to match cell names
Ruiz2021_meta$Patient <- gsub('NH17_', 'NH17-', Ruiz2021_meta$Patient) # change characters to match cell names
Ruiz2021_meta$Cell <- gsub('NH18_', 'NH18-', Ruiz2021_meta$Cell) # change characters to match cell names
Ruiz2021_meta$Patient <- gsub('NH18_', 'NH18-', Ruiz2021_meta$Patient) # change characters to match cell names
Ruiz2021_meta$Cell <- gsub('NH19_', 'NH19-', Ruiz2021_meta$Cell) # change characters to match cell names
Ruiz2021_meta$Patient <- gsub('NH19_', 'NH19-', Ruiz2021_meta$Patient) # change characters to match cell names

In [ ]:
Ruiz2021_meta <- merge(Ruiz2021_meta, Ruiz2021_clinical, by= "Patient")
Ruiz2021_meta$Cell <- paste0(Ruiz2021_meta$Cell, '-1')

In [ ]:
# to include the other good quality cells even they do not have annotated cell type metadata

meta_tmp <- data.frame(Cell = colnames(Ruiz2021_final), Patient = Ruiz2021_final@meta.data$orig.ident)

In [ ]:
Ruiz2021_meta <- list(Ruiz2021_meta, meta_tmp) %>% reduce(full_join) %>%
        mutate(Method = 'nuclei', Platform = '10x_v3')

In [ ]:
Ruiz2021_matrix <- GetAssayData(Ruiz2021_final, slot = 'counts')[,colnames(Ruiz2021_final) %in% Ruiz2021_meta$Cell]

In [ ]:
gene.names <- checkGeneSymbols(rownames(Ruiz2021_matrix), unmapped.as.na=FALSE)
rownames(Ruiz2021_matrix) <- make.unique(gene.names$Suggested.Symbol)

## Merging metadata

In [ ]:
common_names <- intersect(rownames(Yuan2018_matrix), rownames(Neftel2019_10x_matrix))
common_names <- intersect(rownames(Neftel2019_smart_matrix), common_names)
common_names <- intersect(rownames(Wang2019_matrix), common_names)
common_names <- intersect(rownames(Wang2020_matrix), common_names)
common_names <- intersect(rownames(Zhao2020_matrix), common_names)
common_names <- intersect(rownames(Bhaduri2020_matrix), common_names)
common_names <- intersect(rownames(Yu2020_matrix), common_names)
common_names <- intersect(rownames(Wu2020_matrix), common_names)
common_names <- intersect(rownames(Couturier2020_matrix), common_names)
common_names <- intersect(rownames(Johnson2020_matrix), common_names)
common_names <- intersect(rownames(Richards2021_matrix), common_names)
common_names <- intersect(rownames(Darmanis2017_matrix), common_names)
common_names <- intersect(rownames(Sankowski2019_matrix), common_names)
common_names <- intersect(rownames(Goswami2019_matrix), common_names)
common_names <- intersect(rownames(Pombo2021_matrix), common_names)
common_names <- intersect(rownames(Mathewson2021_matrix), common_names)
common_names <- intersect(rownames(Ruiz2021_matrix), common_names)

In [ ]:
cells_all <- cbind(Yuan2018_matrix[rownames(Yuan2018_matrix) %in% common_names,],
                   Neftel2019_10x_matrix[rownames(Neftel2019_10x_matrix) %in% common_names,])                   
cells_all <- cbind(cells_all, Neftel2019_smart_matrix[rownames(Neftel2019_smart_matrix) %in% common_names,])
cells_all <- cbind(cells_all, Wang2019_matrix[rownames(Wang2019_matrix) %in% common_names,])
cells_all <- cbind(cells_all, Wang2020_matrix[rownames(Wang2020_matrix) %in% common_names,])
cells_all <- cbind(cells_all, Zhao2020_matrix[rownames(Zhao2020_matrix) %in% common_names,])
cells_all <- cbind(cells_all, Bhaduri2020_matrix[rownames(Bhaduri2020_matrix) %in% common_names,])
cells_all <- cbind(cells_all, Yu2020_matrix[rownames(Yu2020_matrix) %in% common_names,])
cells_all <- cbind(cells_all, Wu2020_matrix[rownames(Wu2020_matrix) %in% common_names,])
cells_all <- cbind(cells_all, Couturier2020_matrix[rownames(Couturier2020_matrix) %in% common_names,])
cells_all <- cbind(cells_all, Johnson2020_matrix[rownames(Johnson2020_matrix) %in% common_names,])
cells_all <- cbind(cells_all, Richards2021_matrix[rownames(Richards2021_matrix) %in% common_names,])
cells_all <- cbind(cells_all, Darmanis2017_matrix[rownames(Darmanis2017_matrix) %in% common_names,])
cells_all <- cbind(cells_all, Sankowski2019_matrix[rownames(Sankowski2019_matrix) %in% common_names,])
cells_all <- cbind(cells_all, Goswami2019_matrix[rownames(Goswami2019_matrix) %in% common_names,])
cells_all <- cbind(cells_all, Pombo2021_matrix[rownames(Pombo2021_matrix) %in% common_names,])
cells_all <- cbind(cells_all, Mathewson2021_matrix[rownames(Mathewson2021_matrix) %in% common_names,])
cells_all <- cbind(cells_all, Ruiz2021_matrix[rownames(Ruiz2021_matrix) %in% common_names,])

In [ ]:
author <- c(rep('Yuan2018', length(colnames(Yuan2018_matrix))),
              rep('Neftel2019_10x', length(colnames(Neftel2019_10x_matrix))),
              rep('Neftel2019_smart', length(colnames(Neftel2019_smart_matrix))),
              rep('Wang2019', length(colnames(Wang2019_matrix))),
              rep('Wang2020', length(colnames(Wang2020_matrix))),
              rep('Zhao2020', length(colnames(Zhao2020_matrix))),
              rep('Bhaduri2020', length(colnames(Bhaduri2020_matrix))),
              rep('Yu2020', length(colnames(Yu2020_matrix))),
              rep('Wu2020', length(colnames(Wu2020_matrix))),
              rep('Couturier2020', length(colnames(Couturier2020_matrix))),
              rep('Johnson2020', length(colnames(Johnson2020_matrix))),
              rep('Richards2021', length(colnames(Richards2021_matrix))),
              rep('Darmanis2017', length(colnames(Darmanis2017_matrix))),
              rep('Sankowski2019', length(colnames(Sankowski2019_matrix))),
              rep('Goswami2019', length(colnames(Goswami2019_matrix))),
              rep('Pombo2021', length(colnames(Pombo2021_matrix))),
              rep('Mathewson2021', length(colnames(Mathewson2021_matrix))),
              rep('Ruiz2021', length(colnames(Ruiz2021_matrix)))
             )

In [ ]:
meta <- data.frame(Cell = colnames(cells_all), Author = author)

In [ ]:
meta_datasets <- list(Yuan2018_meta, Neftel2019_10x_meta,Neftel2019_smart_meta,Wang2019_meta,Wang2020_meta,
                     Zhao2020_meta,Bhaduri2020_meta,Yu2020_meta,Wu2020_meta,Couturier2020_meta,Johnson2020_meta,
                     Richards2021_meta,Darmanis2017_meta, Sankowski2019_meta,Goswami2019_meta, 
                      Pombo2021_meta, Mathewson2021_meta, Ruiz2021_meta) %>% reduce(full_join)

In [ ]:
meta_all <- list(meta, meta_datasets) %>% reduce(inner_join, by = 'Cell') %>% distinct(Cell, .keep_all = TRUE) %>% 
            column_to_rownames('Cell')

In [ ]:
names(meta_all)[names(meta_all) == "Author"] <- "author"
names(meta_all)[names(meta_all) == "Celltype (malignancy)"] <- "celltype_malignant"
names(meta_all)[names(meta_all) == "Celltype (major-lineage)"] <- "celltype_major"
names(meta_all)[names(meta_all) == "Celltype (minor-lineage)"] <- "celltype_minor"
names(meta_all)[names(meta_all) == "Sample"] <- "sample"
names(meta_all)[names(meta_all) == "Patient"] <- "patient"
names(meta_all)[names(meta_all) == "Celltype (original)"] <- "celltype_original"
names(meta_all)[names(meta_all) == "Age"] <- "age"
names(meta_all)[names(meta_all) == "Sex"] <- "gender"
names(meta_all)[names(meta_all) == "Location"] <- "location"
names(meta_all)[names(meta_all) == "Platform"] <- "platform"
names(meta_all)[names(meta_all) == "Method"] <- "method"
names(meta_all)[names(meta_all) == "Region"] <- "region"
names(meta_all)[names(meta_all) == "Stage"] <- "stage"
names(meta_all)[names(meta_all) == "1p19q"] <- "chr1p19q"

In [ ]:
meta_all$celltype_original = recode(meta_all$celltype_original, 
                                               Astocyte = "Astrocyte", 
                                               Astrocytes = 'Astrocyte',
                                               b_cell = 'B cells',
                                               dendritic_cell = 'DC',
                                               differentiated_tumor = 'Differentiated tumor',
                                               granulocyte = 'Granulocyte',
                                               fibroblast = 'Fibroblast',
                                               Endothelial_cell = 'Endothelial',
                                               endothelial = "Endothelial", 
                                               Immune = 'Immune cell',
                                               myeloid = 'Myeloid',
                                               oligodendrocyte = 'Oligodendrocyte',
                                               pericyte = 'Pericyte',
                                               NormalBrain = 'Normal brain',
                                               Oligodendrocytes = 'Oligodendrocyte',
                                               prolif_stemcell_tumor = 'Prolif stemcell tumor',
                                               stemcell_tumor = 'Stem cell tumor', 
                                               t_cell = 'T cells',
                                               B = 'B cells',
                                               Mast = 'Mast cells',
                                               `NK cells` = 'NK',
                                               `prol. TAM` = 'Proliferating TAM',
                                               `Tcells 1` = 'T cells 1',
                                               `Tcells 2` = 'T cells 2',
                                               `Tcells 3` = 'T cells 3'
                                   )

meta_all$celltype_minor = recode(meta_all$celltype_minor, 
                                               Astocyte = "Astrocyte")

meta_all$gender = recode(meta_all$gender, 
                         F = "Female", 
                         M = 'Male')
meta_all$stage = recode(meta_all$stage, 
                         Recurrence = "Recurrent")

meta_all[ , -which(names(meta_all) %in% c('IDH'))]

meta_all <- meta_all %>% na_if(., 'NA')

meta_all %>% lapply(table)

## Output for Seurat

List of individual Seurat files

In [ ]:
gbm_matrix <- list(Yuan2018_matrix, Neftel2019_10x_matrix,Neftel2019_smart_matrix,Wang2019_matrix,Wang2020_matrix,
                   Zhao2020_matrix,Bhaduri2020_matrix,Yu2020_matrix,Wu2020_matrix,Couturier2020_matrix,
                   Johnson2020_matrix,Richards2021_matrix,Darmanis2017_matrix, Sankowski2019_matrix,Goswami2019_matrix,
                   Pombo2021_matrix, Mathewson2021_matrix, Ruiz2021_matrix)
studies <- c('Yuan2018', 'Neftel2019_10x', 'Neftel2019_smart', 'Wang2019','Wang2020', 
             'Zhao2020', 'Bhaduri2020', 'Yu2020','Wu2020','Couturier2020', 
             'Johnson2020', 'Richards2021', 'Darmanis2017', 'Sankowski2019', 'Goswami2019',
             'Pombo2021', 'Mathewson2021', 'Ruiz2021')
gbm_all <- list()

for (i in 1:length(gbm_matrix)) {
    seu_obj <- CreateSeuratObject(gbm_matrix[[i]], min.features = 0, min.cells = 0,
                                  project = studies[i])
    gbm_all <- append(gbm_all, seu_obj)
}
names(gbm_all) <- studies

In [ ]:
format(object.size(gbm_all), units = "Gb")

In [ ]:
saveRDS(gbm_all, 'data/core_GBmap_seuratobj_list.rds')

## Output for scArches

In [ ]:
scarches <- merge(gbm_all[[1]], y = unlist(gbm_all[2:length(gbm_all)]), project = "GBM", merge.data = T)

In [ ]:
# Output a logical vector for every gene on whether the more than zero counts per cell
# Extract counts
counts <- GetAssayData(object = scarches, slot = "counts") 

# Output a logical vector for every gene on whether the more than zero counts per cell
nonzero <- counts > 0

# Sums all TRUE values and returns TRUE if more than 100 TRUE values per gene
keep_genes <- Matrix::rowSums(nonzero) >= 100

# Only keeping those genes expressed in more than 100 cells
filtered_counts <- counts[keep_genes, ]

# Reassign to filtered Seurat object
filtered_scarches <- CreateSeuratObject(filtered_counts)
filtered_scarches
format(object.size(filtered_scarches), units = "Gb")

In [ ]:
library(DropletUtils)
mtx <- GetAssayData(object = filtered_scarches, slot = "counts") 
write10xCounts('data/core_GBmap_raw_counts.h5',
  mtx,
  barcodes = colnames(mtx),
  gene.id = rownames(mtx),
  gene.symbol = rownames(mtx),
  gene.type = "Gene Expression",
  overwrite = TRUE,
  type = c("HDF5"),
  genome = "unknown",
  version = c("3"),
  chemistry = NULL,
  original.gem.groups = NULL,
  library.ids = NULL)

In [ ]:
write.table(meta_all,
            'data/core_GBmap_metadata.csv',
           row.names = TRUE, col.names = TRUE, quote = FALSE, sep = ',')